In [273]:
writefile("ch12.tex",latex_st) #use after creating a new latex_st1

'Wrote 1092891 characters'

In [131]:
#PJ: This is a lengthy program that parses LaTeX formulas and extracts substrings from latex_st
from IPython.display import display, Math
from provers import *
import time

def readfile(fn):
    fh=open(fn)
    st=fh.read()
    fh.close()
    return st
latex_fn="SPoAchapters.tex"
latex_st=readfile(latex_fn)
print(latex_fn,len(latex_st),'characters')
#latex_fn="SPoAch2-5.tex"
#latex_st+=readfile(latex_fn)
#print("+"+latex_fn,len(latex_st),'characters')
#latex_fn="SPoAch6-9.tex"
#latex_st+=readfile(latex_fn)
#print("+"+latex_fn,len(latex_st),'characters')

def writefile(fn,st):
    fh=open(fn,'w')
    fh.write(st)
    fh.close()
    return "Wrote "+str(len(st))+" characters"

# Signature for input and output (the LaTeX symbols can be changed to agree with other conventions)

# Constant operation symbols
Cons="c"; Cond="d"; Iden="1"; Zero="0"; Bot="\\bot"; Top="\\top"

# Prefix unary operation symbols
Fop="f"; Gop="g"; Lneg="\\sim"; Rneg="-"; Dia="\\diamond"; Box="\\box"; Rtri="\\vartriangleright"; Ltri="\\vartriangleleft"

# Postfix unary operation symbols
Star="^*"; Inv="^{-1}"; Conv="\\smallsmile"; Pri="'"

# Infix binary operation symbols
Meet="\\wedge"; Join="\\vee"; Mult="\\cdot"; Lres="/"; Rres="\\backslash"; Omult="\\odot"; Smul="*";
Add="+"; Oadd="\\oplus"; Comp="\\circ"; Rimp="\\to"; Limp="\\leftarrow"; Ominus="\\ominus"; triR="\\triangleright"; triL="\\triangleleft"

# Infix binary relation symbols
Le="\\le"; Ge="\\ge"; Eq="="; Neq="\\ne"

# First-order logic connectives and quantifiers
And="\\text{ and }"; Or="\\text{ or }"; Imp="\\implies"; Not="\\neg"; Iff="\\iff"; All="\\forall"; Exists="\\exists"

VAR = set(["u","v","w","x","y","z"])|set("x_"+str(i) for i in range(10))|set("y_"+str(i) for i in range(10))
CONST = set([Cons,Cond,Iden,Zero,Bot,Top])
PREFIX = [(Fop,11),(Gop,11),(Lneg,11),(Rneg,11),(Dia,11),(Box,11),(Rtri,11),(Ltri,11),(Not,5)] # (symbol, precedence)
POSTFIX = [(Star,12),(Inv,12),(Conv,12),(Pri,12)]
INFIX = [(Mult,10),(Omult,10),(Comp,10),(Rres,9),(Lres,9),(triR,9),(triL,9),(Add,8),(Oadd,8),(Ominus,8),(Meet,7),(Join,7),
         (Rimp,6),(Limp,6),(Le,5),(Eq,5),(Neq,5),(And,4),(Or,4),(Imp,3),(Iff,2)]
QUANT = [(All,5),(Exists,5)]

# can add further \newcommand macros in the string below
Macros=""  #r"\newcommand{\coimp}{-\!\raisebox{.5pt}{\scriptsize<}\,}"

p9sym={"u":"u","v":"v","w":"w","x":"x","y":"y","z":"z","c":"c","d":"d","1":"1","0":"0","\\bot":"b","\\top":"t","f":"f","g":"g",
       "\\sim":"~","-":"-","\\diamond":"dd","\\box":"bx","\\triangleright":" r ","\\triangleleft":" t ","\\vartriangleright":"tr","\\vartriangleleft":"tl",
       "^*":"'","^{-1}":"i","\\smallsmile":"'","'":"'","\\wedge":"^","\\vee":" v ","\\cdot":"*","/":"/","\\backslash":"\ ",
       "\\odot":"*","*":"*","+":"+","\\oplus":"+","\\ominus":"<-","\\circ":"*","\\to":"->","\\leftarrow":"<-",
       "\\le":"<=","\\ge":">=","=":"=","\\ne":"!=","\\text{ and }":" & ","\\text{ or }":" | ",
       "\\implies":" -> ","\\iff":" <-> ","\\forall":"all","\\exists":"exists"}

opts=["op(700,infix,\"r\")","op(700,infix,\"t\")"]

################## Parser code (can ignore this) #################
# Terms are read using Vaughn Pratt's top-down parsing algorithm #

symbol_table = {}

def wrap(subt, t): # decide when to add parentheses during printing of terms
    return subt.tex() if subt.lbp > t.lbp or len(subt.a)<=1 else "("+subt.tex()+")"

class symbol_base(object):
    a = []
    def __repr__(self): 
        return self.tex()
    def tex(self):
        if len(self.a) == 0: return self.id
        if len(self.a) == 1: 
          if self.id[0]=="^": return wrap(self.a[0],self)+self.id
          return self.id+" "+wrap(self.a[0],self)
        if len(self.a) == 2: 
          return wrap(self.a[0],self)+self.id+(" " if self.id[0]=='\\' else "")+wrap(self.a[1],self)
        return self.id+" "+self.a[0].id+self.a[1].id+self.a[2].id

def symbol(id, bp=0): # identifier, binding power
    if id in symbol_table:
        s = symbol_table[id]    # look symbol up in table
        s.lbp = max(bp, s.lbp)  # update left binding power
    else:
        class s(symbol_base):   # create class for this symbol
            pass
        s.id = id
        s.lbp = bp
        s.nulld = lambda self: self
        symbol_table[id] = s
    return s

def advance(id=None):
    global token
    if id and token.id != id:
        raise SyntaxError("Expected "+id+" got "+token.id)
    token = next()

def nulld(self): # null denotation
    expr = expression()
    advance(")")
    return expr

def infix(id, bp):
    def leftd(self, left): # left denotation
        self.a = [left]
        self.a.append(expression(bp))
        return self
    symbol(id, bp).leftd = leftd

def prefix(id, bp):
    global token
    def nulld(self): # null denotation
        global token
        if token.id != "(":
            self.a = [expression(bp)]
            return self
        else:
            token = next()
            self.a = []
            if token.id != ")":
                while 1:
                    self.a.append(expression())
                    if token.id != ",":
                        break
                    advance(",")
            advance(")")
            return self
    symbol(id, bp).nulld = nulld

def postfix(id, bp):
    def leftd(self,left): # left denotation
        self.a = [left]
        return self
    symbol(id, bp).leftd = leftd

symbol("(").nulld = nulld
symbol(")")
symbol("[").nulld = nulld
symbol("]")
symbol("(end)")

for st in VAR|CONST: symbol(st)
for t in PREFIX: prefix(t[0],t[1])
for t in POSTFIX: postfix(t[0],t[1])
for t in INFIX: infix(t[0],t[1])
for st in VAR:
    for t in QUANT: prefix(t[0]+" "+st,t[1])

def tokenize(st):
    i = 0
    while i<len(st):
        tok = st[i] #read single-letter token
        j = i+1
        if j<len(st) and st[j]=="_": #read subscript
            j+=1
            if st[j]=="{": j+=1
            while j<len(st) and st[j]>='0' and st[j]<='9': j+=1
            if j<len(st) and st[j]=="}": j+=1
            tok = st[i:j]
        elif j<len(st) and st[i]=="^": #read postfix superscript operation
            if st[j]=="{": j+=1
            if st[j]=="-" or st[j]=="*" or st[j]=="\\": j+=1
            if st[j]=="1": j+=1
            if st[j-1]=='\\':
                while j<len(st) and ((st[j]>='a' and st[j]<='z') or (st[j]>='A' and st[j]<='Z')): j+=1
            if j<len(st) and st[j]=="}": j+=1
            tok = st[i:j]
        elif tok=="{":
            tok = st[j]
            j+=1
        if tok=="\\": #read Latex symbol
            while j<len(st) and ((st[j]>='a' and st[j]<='z') or (st[j]>='A' and st[j]<='Z')): j+=1
            if st[i]=="{" and st[j]=="}": j+=1
            tok = st[i:j]
            if tok in ["\\mathbf","\\forall","\\exists"]:
              j+=2
              if j<len(st) and st[j]=="_": #read subscript
                j+=1
                if st[j]=="{": j+=1
                while j<len(st) and st[j]>='0' and st[j]<='9': j+=1
                if j<len(st) and st[j]=="}": j+=1
              tok = st[i:j]
            elif tok=="\\text":
              j+=2
              while j<len(st) and st[j]>='a' and st[j]<='z': j+=1
              j+=1
              if j<len(st) and st[j]=="}": j+=1
              tok = st[i:j]
        i = j
        if tok!=' ':
            symb = symbol_table[tok]
            if not symb: raise SyntaxError("Unknown operator")
            yield symb()
    symb = symbol_table["(end)"]
    yield symb()

def expression(rbp=0):
    global token
    t = token
    token = next()
    left = t.nulld()
    while rbp < token.lbp:
        t = token
        token = next()
        left = t.leftd(left)
    return left

def parse(str): # e.g., t = parse(r"(p\circ q)\lor \mathbf t")
    global token, next
    next = tokenize(str.replace("{\\sim}","\\sim ").replace("{-}","-")).__next__
    token = next()
    return expression()

def show(A, info=True): # display a (list of) formula(s)
  st = A if type(A)==str else repr(A)
  if info==True: display(Math(Macros+st))

########### end of parser #####################################

# code below assumes 'latex_st' contains the survey (SOASn.tex) 

import re
def section(cl,st=latex_st): #find section of the class cl
    cl = cl.replace('\\','\\\\').replace('$','\\$').replace('{','\\{')
    return re.search(r"(\\hypertarget{"+cl+r"}{.*?)\\hypertarget",st,flags=re.DOTALL).group(1)

def chapter(n,st=latex_st): #find chapter number n
    li = re.findall(r"(\\chapter{.*?%%endchapter)",st,flags=re.DOTALL)
    return li[n-1]

def allclasses(st=latex_st): #return list of class names in st
    li=re.findall(r"\\hypertarget{(.*?)}{",st,flags=re.DOTALL)
    print("Number of classes:", len(li))
    return li

def classname(cl): #find long class name of the class cl
    return re.search(r"\\section{.*?:(.*?)}",section(cl),flags=re.DOTALL).group(1)

def sectiontitle(cl): #find full section title of the class cl
    return re.search(r"\\section{(.*?)}",section(cl),flags=re.DOTALL).group(1)

def fulldefinition(cl): #find axioms in full definition of the class cl
    ax=re.search(r"\\begin{fulldefinition}(.*?)\\end{fulldefinition}",section(cl),flags=re.DOTALL)
    if ax==None: return 'none'
    li=re.findall("\$(.*?)\$",ax.group(1),flags=re.DOTALL)
    return [s for s in li if (s.find("=")>0 or s.find("\\le")>0) and s.find('\\langle')==-1 and s.find('\\mathbf')==-1]

def finitemembers(cl,info=False): #find fine spectrum in the class cl
    fs = re.search(r"\\begin{finitemembers}(.*?)\\end{finitemembers}",section(cl),flags=re.DOTALL)
    if info: print(fs)
    li = re.findall("\$(.*?)\$",fs.group(1),flags=re.DOTALL)
    if info: print(li)
    ind = [s.split("_")[1].split("=")[0].strip() for s in li if s.find("_")>=0 and s.find("=")>=0]
    ind = [int(s[1:-1]) if s[0]=="{" else int(s) for s in ind if s.find('n')==-1 and s.find('k')==-1]
    for i in range(min(8,len(ind))): 
        if ind[i]!=i+1: raise Exception("index error at: ", i+1)
    val = [s.split("_")[1].split("=")[1].strip() for s in li if s.find("_")>=0 and s.find("=")>=0]
    return [int(s) for s in val if s!="" and s.find("n")==-1 and s.find("k")==-1]

def subclasses(cl,info=False): #find subclasses of the class cl
    fs = re.search(r"\\begin{subclasses}(.*?)\\end{subclasses}",section(cl),flags=re.DOTALL)
    if info: print(fs)
    return re.findall(r"\\hyperlink{(.*?)}{",fs.group(1),flags=re.DOTALL)

def set_subclasses(cl,sbcli,longname,st=latex_st): #replace subclasses with those in sbcli
    mo = section_mo(cl,st)
    newsbc = r"\\begin{subclasses}\n"+"\n\n".join(r"  \\hyperlink{"+x+"}{"+x+":"+(longname[x] if x in longname.keys() else "")+"}" for x in sbcli)+r"\n\\end{subclasses}"
    newst = re.sub(r"\\begin{subclasses}.*?\\end{subclasses}",newsbc,mo.group(1),flags=re.DOTALL)
    return st[:mo.start(1)]+newst+st[mo.end(1):]

def subclassposet():
    m = re.findall(r"\\hypertarget{(.*?)}{",latex_st,flags=re.DOTALL)
    return {s:subclasses(s) for s in m}

def superclasses(cl,info=False): #find subclasses of the class cl
    fs = re.search(r"\\begin{superclasses}(.*?)\\end{superclasses}",section(cl),flags=re.DOTALL)
    if info: print(fs)
    return re.findall(r"\\hyperlink{(.*?)}{",fs.group(1),flags=re.DOTALL)

def set_superclasses(cl,spcli,longname,st=latex_st): #replace superclasses with those in spcli
    mo = section_mo(cl,st)
    newspc = r"\\begin{superclasses}\n"+"\n\n".join(r"  \\hyperlink{"+x+"}{"+x+":"+(longname[x] if x in longname.keys() else "")+"}" for x in spcli)+r"\n\\end{superclasses}"
    newst = re.sub(r"\\begin{superclasses}.*?\\end{superclasses}",newspc,mo.group(1),flags=re.DOTALL)
    return st[:mo.start(1)]+newst+st[mo.end(1):]

def superclassposet():
    m = re.findall(r"\\hypertarget{(.*?)}{",latex_st,flags=re.DOTALL)
    return {s:superclasses(s) for s in m}

def finitememberslatex(li): #convert list of numbers to fine spectrum in LaTeX
    return ",\n".join("$f_"+(str(i) if i<10 else "{"+str(i)+"}")+" = "+str(li[i-1])+"$" for i in range(1,len(li)+1))+"\n"

def p9out(A): #output formula A in Prover9 format
  if A.a==[]: return p9sym[A.id]
  if A.id[:7] in ["\\forall","\\exists"]:
    return p9sym[A.id[:7]]+" "+A.id[8:]+"("+p9out(A.a[0])+")"
  if len(A.a)==1:
    #if symbol_table[p9sym[A.id]].lbp!=12:
      return p9sym[A.id]+"("+p9out(A.a[0])+")"
    #return "("+p9out(A.a[0])+")"+p9sym[A.id]
  return "("+p9out(A.a[0])+p9sym[A.id]+p9out(A.a[1])+")"

po=["x<=x","x<=y & y<=x -> x=y","x<=y & y<=z -> x<=z"]
msl=["(x^y)^z=x^(y^z)","x^y=y^x","x^x=x","x^y=x<->x<=y"]
jsl=["(x v y)v z=x v(y v z)","x v y=y v x","x v x=x","x v y=y<->x<=y"]
lat=msl+jsl+["x v(x^y)=x","x^(x v y)=x"]
dlat=lat+["x^(y v z)=(x^y)v(x^z)"]
to=lat+["x^y=x | x^y=y"] #["x<=y | y<=x"]
ba=dlat+["x'v x=t","x'^x=b"]
uo=[]
axioms=[po,jsl,msl,lat,dlat,to,ba,uo]

cli = [allclasses(chapter(i)) for i in range(2,10)]
fam = {x:cx for cx in range(8) for x in cli[cx]}

def fd(cl,info=True,f=fam,new_ax=None):
    try:
        ax = [p9out(parse(e)) for e in fulldefinition(cl)]
    except:
        print("############### Error, skipping", cl)
        return []
    ax = [(x[1:-1] if x[0]=='(' and x[-1]==')' else x) for x in ax]
    ch_axioms = axioms[fam[cl]] if new_ax==None else new_ax
    if info: 
        print(ch_axioms+ax)
    return ch_axioms+ax

def finespectrum(cl,n,info=True,f=fam,new_ax=None): 
  # call Prover9 on the translated full definition of cl and find the fine spectrum up to n
    if info: print(cl)
    ax = fd(cl,new_ax)
    if ax==[]: return []
    t = time.time()
    a = [[1]]+[prover9(ch_axioms+ax,[],10000,10000,k,options=opts) for k in range(2,n+1)]
    if info: print("Time: {:.2f}".format(time.time()-t), "sec")
    return [len(x) for x in a]

def section_mo(cl,st=latex_st):
    cl = cl.replace('\\','\\\\').replace('$','\\$').replace('{','\\{')
    return re.search(r"(\\hypertarget{"+cl+r"}{.*?)\\hypertarget",st,flags=re.DOTALL)

def set_finitemembers(cl,li,st=latex_st): #read cl string from latex_st, replace finitemembers entry and return new latex_st
    mo = section_mo(cl,st)
    newst = re.sub(r"\\begin{finitemembers}.*?\\end{finitemembers}",r"\\begin{finitemembers}\n"+finitememberslatex(li)+r"\\end{finitemembers}",mo.group(1),flags=re.DOTALL)
    return st[:mo.start(1)]+newst+st[mo.end(1):]

def set_classname(cl,new_name,st=latex_st): #replace long class name of the class cl
    mo = section_mo(cl,st)
    newst = re.sub(r"\\section{.*?}",r"\\section{"+new_name+r"}",mo.group(1),flags=re.DOTALL)
    return st[:mo.start(1)]+newst+st[mo.end(1):]

ch=jsl
def compareandupdatefs(chax,cl):
    li = finitemembers(cl)
    sli = [x for x in li if x <= 1000]
    n = min(max(len(sli),3),6)
    fs = finespectrum(chax,cl,n,True)
    print(cl,li)
    return li[:n]==fs or fs==[], fs

def sectionnames(st):
    return re.findall(r"\\hypertarget{(.*?)}{",st,flags=re.DOTALL)

#axioms=[[],["x<=y->x=y"],po,jsl,msl,lat,dlat,to,ba,uo]
def checkfs(li,info=False):
    global latex_st
    m=[[] for x in range(10)]
    for ch in li:
        st1=chapter(ch,latex_st)
        m[ch]=sectionnames(st1)
    print([len(x) for x in m])
    for ch in li:
        print(ch,axioms[ch])
        for cl in m[ch]:
            if fulldefinition(cl)!="none":
                fl = compareandupdatefs(axioms[ch-2],cl)
                if not fl[0]:
                    print("***********",fl)
                    latex_st = set_finitemembers(cl,fl[1],latex_st)


def allclassposets(st=latex_st): #return list of tikz diagrams
    return re.findall(r"(\\begin{tikzpicture}\[xscale=1.*?\\end{tikzpicture}\n)",latex_st,flags=re.DOTALL)

def allnodes(st): #return list of node names in st
    li=re.findall(r"\\node\((.*?)\)",st,flags=re.DOTALL)
    print("Number of nodes:", len(li))
    return li

def lowercovers(nd,st): #return list of lowercovers of node name nd in st
    edges=re.search(r"\\draw\("+nd+"\)(.*?);",st,flags=re.DOTALL)
    return re.findall(r"edge.*?\((.*?)\)",edges.group(1),flags=re.DOTALL) if edges!=None else []

def lc2uc(lc):
    uc={x:[] for x in lc}
    for x in lc:
        for y in lc[x]:
            if y in uc: uc[y].append(x)
    return uc

def smallmembers(cl): #return list of small algebras of the class cl
    sm=re.search(r"\\begin{smallmembers}(.*?)\\end{smallmembers}",section(cl),flags=re.DOTALL)
    if sm==None: return 'none'
    li=re.findall(r"\\begin{tikzpicture}(.*?)\\end{tikzpicture}",sm.group(1),flags=re.DOTALL)
    pl=[list(reversed(re.findall(r"\\node\((.*?);", s,flags=re.DOTALL))) for s in li]
    uc=[{int(s[:s.index(")")]):[int(y) for y in re.findall(r"edge.*?\((.*?)\)",s)] for s in x} for x in pl]
    xy=[{int(s[:s.index(")")]):(re.search(r"\((.*?),",s).group(1), re.search(r",(.*?)\)",s).group(1)) for s in x} for x in pl]
    xy=[{i:tuple((int(z) if z.find(".")==-1 else float(z)) for z in xy[n][i]) for i in xy[n]} for n in range(len(pl))]
    c=[[int(s[:s.index(")")]) for s in x if s.find("label=")!=-1] for x in pl]
    return [({'uc':uc[n], 'xy':xy[n]} if c[n]==[] else {'uc':uc[n], 'xy':xy[n], 'c':c[n][0]}) for n in range(len(pl))]

def uc2p9(uc):
    return [(f"{i}<={j}" if j in uc[i] else f"-({i}<={j})") for i in uc for j in uc]

SPoAchapters.tex 1092126 characters
Number of classes: 56
Number of classes: 44
Number of classes: 42
Number of classes: 81
Number of classes: 91
Number of classes: 56
Number of classes: 61
Number of classes: 81


In [146]:
REqMon=['((x*y)*z)=(x*(y*z))', '(x*1)=x', '(1*x)=x', '((x*y)=z) <-> (y=(x\\ z))']
a=p9(REqMon+["x'=x\ 1"],["x*x'=1"],0,1000)

In [147]:
a

[
 Proof(syntax="Prover9", formula_list=[
 [1, 'x * y = z <-> y = x \\ z # label(non_clause)', []],
 [2, "x * x' = 1 # label(non_clause) # label(goal)", []],
 [3, '1 * x = x', []],
 [4, 'x * y = z | x \\ z != y', [1]],
 [5, "x' = x \\ 1", []],
 [6, "c1 * c1' != 1", [2]],
 [7, 'c1 * (c1 \\ 1) != 1', [5, 6]],
 [8, 'x * (1 * (x \\ y)) = y', [4, 3]],
 [9, 'x * (x \\ y) = y', [3, 8]],
 [10, '$F', [9, 7]]])]

In [5]:
len(latex_st)

1092126

In [126]:
fs=finespectrum('BInFL',8)

BInFL
['(x^y)^z=x^(y^z)', 'x^y=y^x', 'x^x=x', 'x^y=x<->x<=y', '(x v y)v z=x v(y v z)', 'x v y=y v x', 'x v x=x', 'x v y=y<->x<=y', 'x v(x^y)=x', 'x^(x v y)=x', 'x^(y v z)=(x^y)v(x^z)', "x'v x=t", "x'^x=b", '~(-(x))=x', '-(~(x))=x', '((x*y)<=z) <-> (y<=~((-(z)*x)))', '((x*y)<=z) <-> (x<=-((y*~(z))))', '((x*y)*z)=(x*(y*z))', '(x*1)=x', '(1*x)=x']
Number of nonisomorphic models of cardinality 2  is  1
No model of cardinality 3
Number of nonisomorphic models of cardinality 4  is  5
No model of cardinality 5


No model of cardinality 6


No model of cardinality 7


Number of nonisomorphic models of cardinality 8  is  25
Time: 9.86 sec


In [127]:
fs

[1, 1, 0, 5, 0, 0, 0, 25]

In [128]:
print(finitememberslatex(fs))

$f_1 = 1$,
$f_2 = 1$,
$f_3 = 0$,
$f_4 = 5$,
$f_5 = 0$,
$f_6 = 0$,
$f_7 = 0$,
$f_8 = 25$



In [597]:
Fld=['((x+y)+z)=(x+(y+z))', '(x+0)=x', '(-(x)+x)=0', '(x+y)=(y+x)', '((x*y)*z)=(x*(y*z))', '(x*1)=x', '(x*y)=(y*x)', '(x*(y+z))=((x*y)+(x*z))', "(x!=0) -> (x*x'=1)","0'=0"]
p9(Fld,[],1000,1000,[6])

Number of nonisomorphic models of cardinality 2  is  1


Number of nonisomorphic models of cardinality 3  is  1
Number of nonisomorphic models of cardinality 4  is  1


Number of nonisomorphic models of cardinality 5  is  1
No model of cardinality 6
Fine spectrum:  [1, 1, 1, 1, 1, 0]


[[],
 [1],
 [
  Model(cardinality = 2, index = 0,
  operations = {
  "+":[
  [0,1],
  [1,0]], 
  "*":[
  [0,0],
  [0,1]], "-":[0, 1], "'":[0, 1]})],
 [
  Model(cardinality = 3, index = 0,
  operations = {
  "+":[
  [0,1,2],
  [1,2,0],
  [2,0,1]], 
  "*":[
  [0,0,0],
  [0,1,2],
  [0,2,1]], "-":[0, 2, 1], "'":[0, 1, 2]})],
 [
  Model(cardinality = 4, index = 0,
  operations = {
  "+":[
  [0,1,2,3],
  [1,0,3,2],
  [2,3,0,1],
  [3,2,1,0]], 
  "*":[
  [0,0,0,0],
  [0,1,2,3],
  [0,2,3,1],
  [0,3,1,2]], "-":[0, 1, 2, 3], "'":[0, 1, 3, 2]})],
 [
  Model(cardinality = 5, index = 0,
  operations = {
  "+":[
  [0,1,2,3,4],
  [1,3,0,4,2],
  [2,0,4,1,3],
  [3,4,1,2,0],
  [4,2,3,0,1]], 
  "*":[
  [0,0,0,0,0],
  [0,1,2,3,4],
  [0,2,1,4,3],
  [0,3,4,2,1],
  [0,4,3,1,2]], "-":[0, 2, 1, 4, 3], "'":[0, 1, 2, 4, 3]})],
 []]

In [522]:
fs=finespectrum('InPocrim',8)

InPocrim
['x<=x', 'x<=y & y<=x -> x=y', 'x<=y & y<=z -> x<=z', '-(-(x))=x', '((x*y)<=z) <-> (y<=-((-(z)*x)))', '((x*y)*z)=(x*(y*z))', '(x*y)=(y*x)', '(x*1)=x', 'x<=1']


Number of nonisomorphic models of cardinality 2  is  1


Number of nonisomorphic models of cardinality 3  is  1


Number of nonisomorphic models of cardinality 4  is  3


Number of nonisomorphic models of cardinality 5  is  3


Number of nonisomorphic models of cardinality 6  is  12


Number of nonisomorphic models of cardinality 7  is  15


Number of nonisomorphic models of cardinality 8  is  73
Time: 4.27 sec


In [523]:
InPocrim=['x<=x', 'x<=y & y<=x -> x=y', 'x<=y & y<=z -> x<=z', '-(-(x))=x', '((x*y)<=z) <-> (y<=-((-(z)*x)))', '((x*y)*z)=(x*(y*z))', '(x*y)=(y*x)', '(x*1)=x', 'x<=1']
a=p9(InPocrim,[],1000,1000,[8])

Number of nonisomorphic models of cardinality 2  is  1


Number of nonisomorphic models of cardinality 3  is  1


Number of nonisomorphic models of cardinality 4  is  3


Number of nonisomorphic models of cardinality 5  is  3


Number of nonisomorphic models of cardinality 6  is  12


Number of nonisomorphic models of cardinality 7  is  15


Number of nonisomorphic models of cardinality 8  is  73
Fine spectrum:  [1, 1, 1, 3, 3, 12, 15, 73]


In [528]:
m4diag(a[8])

1<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: %3 Pages: 1 -->
 
 
 %3 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 2--4 -->
 
 2--4 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 3--5 -->
 
 3--5 
 
 
<!-- 4--0 -->
 
 4--0 
 
 
<!-- 4--3 -->
 
 4--3 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 4--6 -->
 
 4--6 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 4--7 -->
 
 4--7 
 
 
<!-- 5--1 -->
 
 5--1 
 
 
<!-- 6--5 -->
 
 6--5 
 
 
<!-- 7--5 -->
 
 7--5 
 
 
 
 
        2<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: %3 Pages: 1 -->
 
 
 %3 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 2--6 -->
 
 2--6 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 3--5 -->
 
 3--5 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 4--0 -->
 
 4--0 
 
 
<!-- 4--3 -->
 
 4--3 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 5--7 -->
 
 5--7 
 
 
<!-- 6--4 -->
 
 6--4 
 
 
<!-- 7--1 -->
 
 7--1 
 
 
 
 
        3<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: %3 Pages: 1 -->
 
 
 %3 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 2--4 -->
 
 2--4 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 3--7 -->
 
 3--7 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 4--6 -->
 
 4--6 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 5--1 -->
 
 5--1 
 
 
<!-- 6--0 -->
 
 6--0 
 
 
<!-- 6--3 -->
 
 6--3 
 
 
<!-- 7--5 -->
 
 7--5 
 
 
 
 
        4<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: %3 Pages: 1 -->
 
 
 %3 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 2--4 -->
 
 2--4 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 3--5 -->
 
 3--5 
 
 
<!-- 4--3 -->
 
 4--3 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 4--6 -->
 
 4--6 
 
 
<!-- 5--1 -->
 
 5--1 
 
 
<!-- 6--0 -->
 
 6--0 
 
 
<!-- 7--5 -->
 
 7--5 
 
 
 
 
        5<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: %3 Pages: 1 -->
 
 
 %3 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 2--4 -->
 
 2--4 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 3--5 -->
 
 3--5 
 
 
<!-- 4--0 -->
 
 4--0 
 
 
<!-- 4--3 -->
 
 4--3 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 4--6 -->
 
 4--6 
 
 
<!-- 5--1 -->
 
 5--1 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 7--5 -->
 
 7--5 
 
 
 
 
        6<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: %3 Pages: 1 -->
 
 
 %3 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 2--4 -->
 
 2--4 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 3--5 -->
 
 3--5 
 
 
<!-- 4--0 -->
 
 4--0 
 
 
<!-- 4--3 -->
 
 4--3 
 
 
<!-- 6 -->
 
 6 
 

In [0]:
cpo=allclassposets()
print(len(cpo))
tkz="".join(cpo)
nli=allnodes(tkz)
print(len(nli),len(set(nli)))

In [429]:
appdx=readfile('SPoAappendix.tex')
len(appdx)

17031

In [438]:
#def rep(m): return m.group(1)+"&"+m.group(2)+"\\\\"
#st1=re.sub("\^(\[\[.*?)\|(.*?)\|",rep,st1,flags=re.DOTALL)
def rep(m): return '\\\\'+m.group(1)+'\n\\hyperlink{'+m.group(2)+'}{'+m.group(2)+'}&'
appdx1=re.sub(r'\\\\(.*?)\n(.*?)&',rep,appdx,flags=re.DOTALL)
#print(appdx1[:30])

In [437]:
writefile('SPoAappendix.tex',appdx1)

'Wrote 27445 characters'

In [272]:
cls = allclasses()
longname={x:classname(x) for x in cls}
fullname={x:sectiontitle(x) for x in cls}
for x in cls: #x = 'PoUn'
    latex_st = set_subclasses(x, sorted(lcp[x]), longname, latex_st)
    latex_st = set_superclasses(x, sorted(ucp[x]), longname, latex_st)
    latex_st = set_classname(x, fullname[x].replace(':',r'\\index{'+x+'}:'), latex_st)
latex_st = re.sub(r'%\\abbreviation{.*?}\n\n','',latex_st,flags=re.DOTALL)
len(latex_st)

Number of classes: 512


1092891

In [268]:
longname['RtHp']

' Right hoops'

In [257]:
print(section('PoUn',latex_st))

\hypertarget{PoUn}{\section{PoUn\index{PoUn}: Partially ordered unars}}

\begin{definition}
A \emph{partially ordered unar} (also called a \emph{po-unar} for short) is a po-algebra $\mathbf{P}=\langle P,\le, f\rangle$
such that $P$ is a \hyperlink{Pos}{partially ordered set} and $f$ is a unary operation on $P$ that is

order-preserving: $x\le y\implies f(x)\le f(y)$
\end{definition}

\begin{fulldefinition}
$x\le y\implies f(x)\le f(y)$

\end{fulldefinition}

\begin{properties}
\begin{tabular}{|l|l|}\hline
Classtype  &po-variety\\
Universal theory  &Decidable \\
First-order theory  &Undecidable \\
%Amalgamation property  & \\
%Strong amalgamation property  & \\
%Epimorphisms are surjective  & \\
\hline
\end{tabular}
\end{properties}

\begin{finitemembers}
$f_1=1$, 
$f_2=6$, 
$f_3=43$, 
$f_4=452$
\end{finitemembers}

\begin{subclasses}
  \hyperlink{GalPos}{GalPos: Galois posets}

  \hyperlink{JUn}{JUn: Join-semilattice-ordered unars}

  \hyperlink{MUn}{MUn: Meet-semilattice-ordered unars

In [230]:
#[longname[x] for x in cls[:20]]
lcp['Pos']

['pPos', 'PoUn', 'PoNUn', 'PoMag', 'PoImpA', 'Jslat', 'Mslat', 'Set']

In [214]:
ucp=lc2uc(scp)

In [216]:
lcp=lc2uc(ucp)

In [217]:
[set(lcp[x])-set(scp[x]) for x in lcp if set(lcp[x])-set(scp[x])!=set()]

[]

In [219]:
#[len(set(scp[x])-set(lcp[x])) for x in lcp] 

In [213]:
#subclassposet
scp={'Pos': ['pPos', 'PoMag', 'PoImpA', 'PoNUn', 'PoUn', 'Jslat', 'Mslat', 'Set'],
 'pPos': ['PoMon', 'pJslat', 'pMslat', 'pSet'],
 'PoUn': ['GalPos', 'RPoUn', 'JUn', 'MUn', 'Unar'],
 'PoNUn': ['GalPos', 'JNUn', 'MNUn'],
 'PoMag': ['LrPoMag', 'PoSgrp', 'JMag', 'MMag'],
 'PoSgrp': ['CPoSgrp', 'IdPoSgrp', 'LrPoSgrp', 'PoMon', 'JSgrp', 'MSgrp'],
 'PoMon': ['CPoMon', 'IdPoMon', 'IPoMon', 'LrPoMon', 'JMon', 'MMon'],
 'IPoMon': ['CIPoMon', 'Polrim', 'IJMon', 'IMMon'],
 'IdPoSgrp': ['CIdPoSgrp', 'IdLrPoSgrp', 'IdPoMon', 'IdJSgrp', 'IdMSgrp'],
 'IdPoMon': ['CIdPoMon', 'IdLrPoMon', 'IdJMon', 'IdMMon'],
 'PoImpA': ['DivPos', 'LrPoMag', 'JImpA', 'MImpA'],
 'LrPoMag': ['RPoMag', 'LrPoSgrp', 'LrJMag', 'LrMMag'],
 'LrPoSgrp': ['RPoMon', 'LrPoMon', 'IdLrPoSgrp', 'RPoSgrp', 'LrJSgrp', 'LrMSgrp'],
 'LrPoMon': ['RPoMon', 'IdLrPoMon', 'Polrim', 'LrJMon', 'LrMMon'],
 'Polrim': ['Porim', 'ILrJMon', 'ILrMMon'],
 'IdLrPoSgrp': ['IdRPoSgrp', 'IdLrPoMon', 'IdLrJSgrp', 'IdLrMSgrp'],
 'IdLrPoMon': ['IdRPoMon', 'IdLrJMon', 'IdLrMMon'],
 'RPoUn': ['InPoMon', 'RJUn', 'RMUn'],
 'DivPos': ['CDivPos', 'RPoMag', 'DivJslat', 'DivMslat'],
 'RPoMag': ['CRPoMag', 'RPoSgrp', 'InPoMag', 'RJMag', 'RMMag'],
 'RPoSgrp': ['CRPoSgrp', 'RPoMon', 'IdRPoSgrp', 'InPoSgrp', 'RJSgrp', 'RMSgrp'],
 'RPoMon': ['CRPoMon', 'InPoMon', 'IdRPoMon', 'Porim', 'RJMon', 'RMMon'],
 'Porim': ['InPorim', 'Pocrim', 'IRJMon', 'IRMMon'],
 'IdRPoSgrp': ['CIdRPoSgrp', 'IdRPoMon', 'IdInPoSgrp', 'IdRJSgrp', 'IdRMSgrp'],
 'IdRPoMon': ['CIdRPoMon', 'IdInPoMon', 'IdRJMon', 'IdRMMon'],
 'GalPos': ['InPos', 'GalJslat', 'GalMslat'],
 'InPos': ['InPoMag', 'InLat'],
 'InPoMag': ['CyInPoMag', 'InPoSgrp', 'InLMag'],
 'InPoSgrp': ['CyInPoSgrp', 'InPoMon', 'IdInPoSgrp', 'InLSgrp'],
 'InPoMon': ['CyInPoMon', 'IInPoMon', 'PoGrp', 'IdInPoMon', 'InPorim', 'InLMon'],
 'InPorim': ['CyInPorim', 'IInLMon'],
 'CyInPoMag': ['CInPoMag', 'CyInPoSgrp', 'CyInLMag'],
 'CyInPoSgrp': ['CInPoSgrp', 'CyInPoMon', 'CyIdInPoSgrp', 'CyInLSgrp'],
 'CyInPoMon': ['CInPoMon', 'CyInPorim', 'CyIdInPoMon', 'PoGrp', 'CyInLMon'],
 'CyInPorim': ['InPocrim', 'CyIInLMon'],
 'PoGrp': ['AbPoGrp', 'LGrp'],
 'CPoSgrp': ['CIdPoSgrp', 'CRPoSgrp', 'CPoMon', 'CJSgrp', 'CMSgrp'],
 'CPoMon': ['CIdPoMon', 'CIPoMon', 'CRPoMon', 'CJMon', 'CMMon'],
 'CIPoMon': ['Pocrim', 'CIJMon', 'CIMMon'],
 'CIdPoSgrp': ['CIdRPoSgrp', 'CIdPoMon', 'CIdJSgrp', 'CIdMSgrp'],
 'CIdPoMon': ['CIdRPoMon', 'CIdJMon', 'CIdMMon'],
 'CDivPos': ['CRPoMag', 'BCK', 'CDivJslat', 'CDivMslat'],
 'BCK': ['Pocrim', 'HilA', 'BCKJslat', 'BCKMslat'],
 'CRPoMag': ['CRPoSgrp', 'CInPoMag', 'CRJMag', 'CRMMag'],
 'HilA': ['TarA'],
 'TarA': [],
 'CRPoSgrp': ['CInPoSgrp', 'CRPoMon', 'CIdRPoSgrp', 'CRJSgrp', 'CRMSgrp'],
 'CRPoMon': ['CInPoMon', 'Pocrim', 'CIdRPoMon', 'CRJMon', 'CRMMon'],
 'Pocrim': ['InPocrim', 'CIRJMon', 'CIRMMon'],
 'CIdRPoSgrp': ['CIdRPoMon', 'CIdInPoSgrp', 'CIdRJSgrp', 'CIdRMSgrp'],
 'CIdRPoMon': ['IdRPocrim', 'CIdInPoMon', 'CIdRJMon', 'CIdRMMon'],
 'CInPoMag': ['CInPoSgrp', 'CInLMag'],
 'CInPoSgrp': ['CInPoMon', 'CIdInPoSgrp', 'CInLSgrp'],
 'CInPoMon': ['InPocrim', 'AbPoGrp', 'CIdInPoMon', 'CInLMon'],
 'InPocrim': ['CIInLMon'],
 'AbPoGrp': ['AbLGrp'],

 'Jslat': ['Slat$_1$', 'Slat$_0$', 'pJslat', 'JUn', 'JNUn', 'JMag', 'JImpA', 'Lat'],
 'pJslat': ['JMon', 'ubJslat', 'lbJslat', 'pLat'],
 'lbJslat': ['IdSrng$_0$', 'bLat'],
 'ubJslat': ['IJMon', 'bLat'],
 'JUn': ['GalJslat', 'RJUn', 'LUn'],
 'JNUn': ['GalJslat', 'LNUn'],
 'GalJslat': ['GalLat'],
 'JMag': ['JSgrp', 'LrJMag', 'LMag'],
 'JSgrp': ['CJSgrp', 'JMon', 'IdJSgrp', 'LrJSgrp', 'LSgrp'],
 'JMon': ['CJMon', 'LrJMon', 'IdJMon', 'IJMon', 'IdSrng$_{01}$', 'LMon'],
 'IdSrng$_0$': ['IdSrng$_{01}$'], 
 'IdSrng$_{01}$': ['KA'],
 'KA': ['ActA', 'KLat'],
 'IJMon': ['CIJMon', 'ILrJMon', 'ILMon'],
 'IdJSgrp': ['CIdJSgrp', 'IdJMon', 'IdLrJSgrp', 'IdLSgrp'],
 'IdJMon': ['CIdJMon', 'IdLrJMon', 'IdLMon'],
 'JImpA': ['LrJMag', 'DivJslat', 'LImpA'],
 'LrJMag': ['RJMag', 'LrJSgrp', 'LrLMag'],
 'LrJSgrp': ['RJMon', 'LrJMon', 'IdLrJSgrp', 'RJSgrp', 'LrLSgrp'],
 'LrJMon': ['RJMon', 'ILrJMon', 'IdLrJMon', 'LrLMon'],
 'ILrJMon': ['IRJMon', 'ILrLMon'],
 'IdLrJSgrp': ['IdRJSgrp', 'IdLrJMon', 'IdLrLSgrp'],
 'IdLrJMon': ['IdRJMon', 'IdLrLMon'],
 'RJUn': ['InJMon', 'RLUn'],
 'DivJslat': ['CDivJslat', 'RJMag', 'DivLat'],
 'RJMag': ['CRJMag', 'RJSgrp', 'RLMag'],
 'RJSgrp': ['CRJSgrp', 'RJMon', 'IdRJSgrp', 'RLSgrp'],
 'RJMon': ['CRJMon', 'IdRJMon', 'IRJMon', 'RLMon'],
 'IRJMon': ['CIRJMon', 'IRLMon'],
 'IdRJSgrp': ['CIdRJSgrp', 'IdRJMon', 'IdRLSgrp'],
 'IdRJMon': ['CIdRJMon', 'IdRLMon'],
 'CJSgrp': ['CJMon', 'CRJSgrp', 'CIdJSgrp', 'CLSgrp'],
 'CJMon': ['CIJMon', 'CRJMon', 'CIdJMon', 'CLMon'],
 'CIJMon': ['CIRJMon', 'CILMon'],
 'CIdJSgrp': ['CIdJMon', 'CIdRJSgrp', 'CIdLSgrp'],
 'CIdJMon': ['CIdRJMon', 'CIdLMon'],
 'CDivJslat': ['CRJMag', 'BCKJslat', 'CDivLat'],
 'BCKJslat': ['BCKLat'],
 'CRJMag': ['CRJSgrp', 'CInJMag', 'CRLMag'],
 'CRJSgrp': ['CInJSgrp', 'CRJMon', 'CIdRJSgrp', 'CRLSgrp'],
 'CRJMon': ['CIRJMon', 'CIdRJMon', 'CRLMon'],
 'CIRJMon': ['IdCIRJMon', 'CIRLMon'],
 'CIdRJSgrp': ['CIdRJMon', 'CIdRLSgrp'],
 'CIdRJMon': ['IdRCIRJMon', 'CIdRLMon'],

 'Mslat': ['pMslat', 'MMag', 'MImpA', 'MUn', 'MNUn', 'Lat'],
 'pMslat': ['MMon', 'pLat'],
 'MUn': ['GalMslat', 'RMUn', 'LUn'],
 'MNUn': ['GalMslat', 'LNUn'],
 'GalMslat': ['GalLat'],
 'MMag': ['MSgrp', 'LrMMag', 'LMag'],
 'MSgrp': ['CMSgrp', 'IdMSgrp', 'LrMSgrp', 'LSgrp', 'MMon'],
 'MMon': ['CMMon', 'IdMMon', 'IMMon', 'LrMMon', 'LMon'],
 'IMMon': ['CIMMon', 'ILrMMon', 'ILMon'],
 'IdMSgrp': ['CIdMSgrp', 'IdLrMSgrp', 'IdMMon', 'IdLSgrp'],
 'IdMMon': ['CIdMMon', 'IdLrMMon', 'IdLMon'],
 'MImpA': ['DivMslat', 'LrMMag', 'LImpA'],
 'LrMMag': ['RMMag', 'LrMSgrp', 'LrLMag'],
 'LrMSgrp': ['IdLrMSgrp', 'LrMMon', 'RMSgrp', 'LrLSgrp'],
 'LrMMon': ['IdLrMMon', 'ILrMMon', 'RMMon', 'LrLMon'],
 'ILrMMon': ['IRMMon', 'ILrLMon', 'RtHp'],
 'RtHp': ['Hp'],
 'IdLrMSgrp': ['IdRMSgrp', 'IdLrMMon', 'IdLrLSgrp'],
 'IdLrMMon': ['IdRMMon', 'IdLrLMon'],
 'RMUn': ['InMMon', 'RLUn'],
 'DivMslat': ['CDivMslat', 'RMMag', 'DivLat'],
 'RMMag': ['CRMMag', 'RMSgrp', 'RLMag'],
 'RMSgrp': ['CRMSgrp', 'IdRMSgrp', 'RMMon', 'RLSgrp'],
 'RMMon': ['IdRMMon', 'IRMMon', 'CRMMon', 'RLMon'],
 'IRMMon': ['CIRMMon', 'IRLMon'],
 'IdRMSgrp': ['CIdRMSgrp', 'IdRMMon', 'IdRLSgrp'],
 'IdRMMon': ['CIdRMMon', 'IdRLMon'],
 'CMSgrp': ['CMMon', 'CIdMSgrp', 'CRMSgrp', 'CRLSgrp', 'CLSgrp'],
 'CMMon': ['CIMMon', 'CRMMon', 'CIdMMon', 'CLMon'],
 'CIMMon': ['CIRMMon', 'CILMon'],
 'CIdMSgrp': ['CIdMMon', 'CIdRMSgrp', 'CIdLSgrp'],
 'CIdMMon': ['CIdRMMon', 'CIdLMon'],
 'CDivMslat': ['CRMMag', 'BCKMslat', 'CDivLat'],
 'BCKMslat': ['BCKLat'],
 'CRMMag': ['CRMSgrp', 'CInMMag', 'CRLMag'],
 'CRMSgrp': ['CIdRMSgrp', 'CRMMon', 'CRLSgrp'],
 'CRMMon': ['CIdRMMon', 'CIRMMon', 'CRLMon'],
 'CIRMMon': ['CIRL', 'CIRLMon'],
 'Hp': ['WaHp', 'BrSlat', 'CGBL'],
 'CIdRMSgrp': ['CIdRMMon', 'CIdRLSgrp'],
 'CIdRMMon': ['IdRCIRMMon', 'CIdRLMon'],
 'BrSlat': ['BrA'],

 'Lat': ['pLat', 'LMag', 'LImpA', 'LNUn', 'LUn', 'ModLat', 'JsdLat','MsdLat', 'OLat'],
 'pLat': ['LMon', 'pDLat'],
 'bLat': ['bModLat', 'CplmLat', 'bDLat'],
 'LUn': ['RLUn', 'DLUn'],
 'LNUn': ['GalLat', 'DLNUn'],
 'LMag': ['LrLMag', 'LSgrp', 'DLMag', 'MultLat'],
 'LSgrp': ['CLSgrp', 'LrLSgrp', 'LMon', 'IdLSgrp', 'DLSgrp'],
 'LMon': ['CLMon', 'ILMon', 'LrLMon', 'IdLMon', 'DLMon'],
 'KLat': ['ActLat'],
 'ActLat': ['TrivA'],
 'ModLat': ['DLat', 'ComModLat'],
 'MultLat': ['LSgrp'],
 'ILMon': ['CILMon', 'ILrLMon', 'DILMon'],
 'IdLSgrp': ['CIdLSgrp', 'IdLrLSgrp', 'IdLMon', 'DIdLSgrp'],
 'IdLMon': ['CIdLMon', 'IdLrLMon', 'DIdLMon'],
 'LImpA': ['DivLat', 'LrLMag', 'DLImpA'],
 'LrLMag': ['RLMag', 'LrLSgrp', 'DLrLMag'],
 'LrLSgrp': ['R$ell$Sgrp', 'LrLMon', 'IdLrLSgrp', 'RLSgrp', 'DLrLSgrp'],
 'LrLMon': ['ILrLMon', 'RL', 'IdLrLMon', 'DLrLMon'],
 'ILrLMon': ['IRL', 'DILrLMon'],
 'IdLrLSgrp': ['IdRLSgrp', 'IdLrLMon', 'DIdLrLSgrp'],
 'IdLrLMon': ['IdRL', 'DIdLrLMon'],
 'RLUn': ['InLMon', 'DRLUn'],
 'DivLat': ['CDivLat', 'RLMag', 'DDivLat'],
 'RLMag': ['CRLMag', 'InLMag', 'RLSgrp', 'DRLMag'],
 'RLSgrp': ['CRLSgrp', 'InLSgrp', 'RL', 'IdRLSgrp', 'DRLSgrp'],
 'RL': ['bRL', 'CRL', 'IRL', 'IdRL', 'DRL', 'ActLat', 'CanRL', 'FL'],
 'bRL': ['ILLA', 'MALLA'],
 'IRL': ['CIRL', 'IInFL', 'DIRL'],
 'IdRLSgrp': ['CIdRLSgrp', 'IdInLSgrp', 'IdRL', 'DIdRLSgrp'],
 'IdRL': ['CIdRL', 'IdInFL', 'DIdRL'],
 'FL': ['bRL', 'FL$_e$', 'FL$_w$', 'FL$_c$', 'DFL', 'InFL'],
 'FL$_c$': ['FL$_{ec}$', 'DFL$_c$'],
 'FL$_e$': ['FL$_{ec}$', 'FL$_{ew}$', 'DFL', 'DFL$_e$', 'ILLA'],
 'FL$_w$': ['DFL$_w$'],
 'FL$_{ec}$': ['DFL$_{ec}$'],
 'FL$_{ew}$': ['DFL$_{ew}$'],
 'GalLat': ['InLat', 'DGalLat'],
 'InLat': ['InLMag', 'DInLat', 'Bilat'],
 'InLMag': ['CyInLMag', 'InLSgrp', 'DInLMag'],
 'InLSgrp': ['CyInLSgrp', 'InFL', 'IdInLSgrp', 'DInLSgrp'],
 'InFL': ['CyInFL', 'IInFL', 'IdInFL', 'DInFL'],
 'IInFL': ['CyIInFL', 'DIInFL'],
 'CyInLMag': ['CInLMag', 'CyInLSgrp', 'CyDInLMag'],
 'CyInLSgrp': ['CInLSgrp', 'CyInFL', 'CyIdInLSgrp', 'CyDInLSgrp'],
 'CyInFL': ['CInFL', 'CyIInFL', 'CyIdInFL', 'CyDInFL'],
 'CyIInFL': ['CIInFL', 'CyDIInFL'],
 'CLSgrp': ['CRLSgrp', 'CLMon', 'CIdLSgrp', 'CDLSgrp'],
 'CLMon': ['CILMon', 'CRL', 'CIdLMon', 'CDLMon'],
 'CILMon': ['CIRL', 'CDILMon'],
 'CIdLSgrp': ['CIdRLSgrp', 'CIdLMon', 'CDIdLSgrp'],
 'CIdLMon': ['CIdRL', 'CDIdLMon'],
 'CDivLat': ['CRLMag', 'CDDivLat'],
 'BCKLat': ['HA'],
 'CRLMag': ['CInLMag', 'CRLSgrp', 'CDRLMag'],
 'CRLSgrp': ['CInLSgrp', 'CRL', 'CIdRLSgrp', 'CDRLSgrp'],
 'CRL': ['CIRL', 'CInFL', 'CIdRL', 'CDRL'],
 'CIRL': ['CIInFL', 'CDIRL'],
 'CIdRLSgrp': ['CIdInLSgrp', 'CIdRL', 'CDIdRLSgrp'],
 'CIdRL': ['CIdInFL', 'CDIdRL'],
 'CIdInFL': ['CDIdInFL'],
 'CInLMag': ['CInLSgrp', 'CDInLMag'],
 'CInLSgrp': ['CInFL', 'CIdInLSgrp', 'CDInLSgrp'],
 'CInFL': ['CIInFL', 'CIdInFL', 'CDInFL', 'MALLA'],
 'CIInFL': ['CDIInFL'],
 'JsdLat': ['SdLat'],
 'MsdLat': ['SdLat'],
 'SdLat': ['NdLat'],
 'NdLat': ['ADLat'],
 'ADLat': ['DLat'],
 'CplmLat': ['CplmModLat'],
 'OLat': ['OModLat'],
 'OModLat': ['ModOLat'],
 'ModOLat': ['BA'],
 'SkLat': ['Lat', 'RecBnd'],
 'Bilat': ['TrivA'], # 'DBilat'],
 'CanRL': ['TrivA'],
 'CplmModLat': ['BA'],
 'FRng': ['TrivA'],
 'ILLA': ['LLA'],
 'LLA': ['TrivA'],
 'MALLA': ['LLA'],

 'DLat': ['bDLat', 'pDLat', 'DLUn', 'DLNUn', 'DLMag', 'DLImpA', 'ToLat', 'BA', 'pcDLat'],
 'pDLat': ['DLMon', 'bDLat', 'pToLat', 'pBA'],
 'bDLat': ['BA', 'bToLat', 'DpAlg', 'DdpAlg', 'BoolLat', 'OckA'],
 'DLUn': ['DGalLat', 'DRLUn', 'ToUn', 'BUn'],
 'DLNUn': ['DGalLat', 'ToNUn', 'BNUn', 'OckA'],
 'pcDLat': ['DpAlg'],
 'OckA': ['DmA'],
 'DmA': ['KLA', 'LA$_n$'],
 'DmMon': [],
 'DpAlg': ['DDbpAlg', 'StAlg'],
 'DdpAlg': ['DDblpAlg'],
 'DDblpAlg': ['DblStAlg'],
 'StAlg': ['DblStAlg'],
 'DblStAlg': ['BA'],
 'DLMag': ['DLSgrp', 'DLrLMag', 'ToMag', 'BMag'],
 'DLSgrp': ['CDLSgrp', 'DIdLSgrp', 'DLMon', 'DLrLSgrp', 'ToSgrp', 'BSgrp'],
 'DLMon': ['CDLMon', 'DIdLMon', 'DILMon', 'DLrLMon', 'ToMon', 'BMon'],
 'DILMon': ['ALat', 'CDILMon', 'DILrLMon', 'IToMon', 'BIMon'],
 'DIdLSgrp': ['CDIdLSgrp', 'DIdLMon', 'DIdLrLSgrp', 'IdToSgrp', 'BIdSgrp'],
 'DIdLMon': ['CDIdLMon', 'DIdLrLMon', 'IdToMon', 'BIdMon'],
 'DLImpA': ['CDLSgrp', 'LSgrp', 'DLrLMag', 'DDivLat', 'ToImpA', 'BImpA', 'ImpLat'],
 'DLrLMag': ['DRLMag', 'DLrLSgrp', 'LrToMag', 'BLrMag'],
 'DLrLSgrp': ['DR$ell$Sgrp', 'DLrLMon', 'DIdLrLSgrp', 'DRLSgrp', 'LrToSgrp', 'BLrSgrp'],
 'DLrLMon': ['DRL', 'DILrLMon', 'DIdLrLMon', 'LrToMon', 'BILrMon'],
 'DILrLMon': ['DIRL', 'ILrToMon', 'BIdLrSgrp'],
 'DIdLrLSgrp': ['DIdRLSgrp', 'DIdLrLMon', 'IdLrToSgrp', 'BIdLrMon'],
 'DIdLrLMon': ['DIdRL', 'IdLrToMon', 'BRUn'],
 'DRLUn': ['DInLMon', 'RToUn', 'BDivLat'],
 'DDivLat': ['DRLMag', 'CDDivLat', 'ToDivLat', 'BRMag'],
 'DRLMag': ['CDRLMag', 'DRLSgrp', 'DInLMag', 'RToMag', 'BRSgrp'],
 'DRLSgrp': ['CDRLSgrp', 'DRL', 'DIdRLSgrp', 'DInLSgrp', 'RToSgrp', 'BRL'],
 'DRL': ['CDRL', 'DFL$_e$', 'DIdRL', 'DIRL', 'DInFL', 'RToMon', 'BIRL', 'GBL'],
 'DIRL': ['CDIRL', 'DIInFL', 'IRToMon', 'BIdRSgrp'],
 'DIdRLSgrp': ['CDIdRLSgrp', 'DIdRL', 'DIdInLSgrp', 'IdRToSgrp', 'BIdRL'],
 'DIdRL': ['CDIdRL', 'DIdInFL', 'IdRToMon', 'BGalA'],
 'DGalLat': ['DInLat', 'GalToLat', 'BGalLat'],
 'DInLat': ['DInLMag', 'InToLat', 'BInMag'],
 'DInLMag': ['CyDInLMag', 'DInLSgrp', 'InToMag', 'BInSgrp'],
 'DInLSgrp': ['CyDInLSgrp', 'DInFL', 'DIdInLSgrp', 'InToSgrp', 'BInFL'],
 'DInFL': ['CyDInFL', 'DIInFL', 'DLGrp', 'DIdInFL', 'BIInFL'],
 'DIInFL': ['CyDIInFL', 'BCyInMag', 'psMV'],
 'CyDInLMag': ['CDInLMag', 'CyDInLSgrp', 'CyInToMag', 'BCyInSgrp'],
 'CyDInLSgrp': ['CDInLSgrp', 'CyDInFL', 'CyDIdInLSgrp',
 'CyInToSgrp', 'BCyInFL'],
 'CyDInFL': ['CDInFL', 'CyDIInFL', 'CyDIdInFL', 'LGrp', 'BCyIInFL'],
 'CyDIInFL': ['CDIInFL'],
 'LGrp': ['NVLGrp'],
 'RepLGrp': ['AbLGrp', 'ToGrp'],
 'CDLSgrp': ['CDIdLSgrp', 'CDLMon', 'CDRLSgrp', 'CToSgrp', 'BCMon'],
 'CDLMon': ['ALat', 'CDIdLMon', 'CDILMon', 'CDRL', 'CToMon', 'BCIMon'],
 'CDILMon': ['ALat', 'CDIRL', 'CIToMon', 'BCIdSgrp'],
 'CDIdLSgrp': ['CDIdLMon', 'CDIdRLSgrp', 'CIdToSgrp', 'BCIdMon'],
 'CDIdLMon': ['CDIdRL', 'CIdToMon', 'BCDivLat'],
 'CDDivLat': ['CDRLMag', 'CivToLat', 'BCRMag'],
 'CDRLMag': ['CDRLSgrp', 'CDInLMag', 'CRToMag'],
 'CDRLSgrp': ['CDInLSgrp', 'CDRL', 'CDIdRLSgrp', 'CRSlSgrp'],
 'CDRL': ['CDIRL', 'CDIdRL', 'CDInFL', 'CRSlMon', 'DunnMon'],
 'CDIRL': ['DICDIRL', 'CDIInFL', 'CIRSlMon'],
 'CDIdRLSgrp': ['CDIdRL', 'CDIdInLSgrp', 'CIdRSlSgrp'],
 'CDIdRL': ['DIdRCIRL', 'CDIdInFL', 'CIdRSlMon', 'BCInMag'],
 'CDInLMag': ['CDInLSgrp', 'CInToMag'],
 'CDInLSgrp': ['CDInFL', 'CDIdInLSgrp', 'CInSlSgrp'],
 'CDInFL': ['CDIInFL', 'AbLGrp', 'CDIdInFL', 'CInSlMon', 'DmMon'],
 'CDIInFL': ['CIInSlMon'],
 'AbLGrp': ['AbToGrp', 'LRng'],
 'GBL': ['BLA', 'GBLChn', 'GMV'],
 'GMV': ['MV', 'GMVChn'],
 'psMV': ['MV'],
 'WaHp': ['MV'],
 'BrA': ['HA', 'GBA'],
 'GBA': ['BA'],
 'BoolLat': ['BA'],
 'CRSlSgrp': ['CInSlSgrp', 'CIdRSlSgrp', 'CRSlMon', 'CRToSgrp', 'BCRMon'],
 'CRSlMon': ['CIdRSlMon', 'CIRSlMon', 'CInSlMon', 'CRToMon', 'BCIRMon'],
 'CIRSlMon': ['IMTL', 'CIRToMon', 'CIInSlMon', 'MTLA'],
 'MTLA': ['BLA', 'MTLChn'],
 'BLA': ['MV', 'HA', 'BLChn'],
 'MV': ['BA', 'MVChn'],
 'HA': ['GödA'],
 'GödA': ['BA', 'GödChn', 'BCIdRSgrp'],
 'CIdRSlSgrp': ['CIdInSlSgrp', 'CIdRSlMon', 'CIdRToSgrp', 'BCIdRMon'],
 'CIdRSlMon': ['CIdInSlMon', 'CIdRToMon', 'BCInSgrp'],
 'CInSlSgrp': ['CIdInSlSgrp', 'CInSlMon', 'CInToSgrp', 'BCInMon'],
 'CInSlMon': ['CIdInSlMon', 'IMTL', 'AbLGrp', 'CInToMon', 'CIInSlMon'],
 'DunnMon': ['CDIdRL', 'BCIInRMon', 'DmMon'],
 'IMTL': ['IMTLChn', 'CIInRToMon'],
 'ImpLat': ['GodA', 'MV', 'LGrp'],
 'KLA': ['BA'],
 'NVLGrp': ['RepLGrp'],
 'LA$_n$': ['BA'],
 'LRng': ['CLRng', 'ToRng', 'FRng'],
 'CLRng': ['CToRng'],

 'ToLat': ['pDLat', 'ToMag', 'ToImpA', 'ToNUn', 'ToUn', 'pToLat'],
 'pToLat': ['ToMon'],
 'ToMag': ['ToSgrp', 'LrToMag'],
 'ToSgrp': ['CToSgrp', 'IdToSgrp', 'LrToSgrp', 'ToMon'],
 'ToMon': ['CToMon', 'IdToMon', 'IToMon', 'LrToMon'],
 'IToMon': ['CIToMon', 'ILrToMon'],
 'IdToSgrp': ['CIdToSgrp', 'IdLrToSgrp', 'IdToMon'],
 'IdToMon': ['CIdToMon', 'IdLrToMon'],
 'ToImpA': ['ToDivLat', 'LrToMag'],
 'LrToMag': ['RToMag', 'LrToSgrp'],
 'LrToSgrp': ['IdLrToSgrp', 'RToSgrp', 'LrToMon'],
 'LrToMon': ['IdLrToMon', 'ILrToMon', 'RToMon'],
 'ILrToMon': ['IRToMon'],
 'IdLrToSgrp': ['IdRToSgrp', 'IdLrToMon'],
 'IdLrToMon': ['IdRToMon'],
 'RToUn': ['InToMon'],
 'ToDivLat': ['CToDivLat', 'RToMag'],
 'RToMag': ['CRToMag', 'InToMag', 'RToSgrp'],
 'RToSgrp': ['CRToSgrp', 'IdRToSgrp', 'InToSgrp', 'RToMon'],
 'RToMon': ['CRToMon', 'IdRToMon', 'IRToMon', 'InToMon'],
 'IRToMon': ['CIRToMon', 'IInRToMon'],
 'IdRToSgrp': ['CIdRToSgrp', 'IdInToSgrp', 'IdRToMon'],
 'IdRToMon': ['CIdRToMon', 'IdInToMon'],
 'ToUn': ['RToUn'],
 'ToNUn': ['GalToLat'],
 'GalToLat': ['InToLat'],
 'InToLat': ['InToMag'],
 'InToMag': ['CyInToMag', 'InToSgrp'],
 'InToSgrp': ['CyInToSgrp', 'IdInToSgrp', 'InToMon'],
 'InToMon': ['CyInToMon', 'IdInToMon', 'IInRToMon'],
 'IInRToMon': ['CyIInRToMon'],
 'CyInToMag': ['CInToMag', 'CyInToSgrp'],
 'CyInToSgrp': ['CInToSgrp', 'CyIdInToSgrp', 'CyInToMon'],
 'CyInToMon': ['CInToMon', 'CyIdInToMon', 'CyIInRToMon', 'ToGrp'],
 'CyIInRToMon': ['IMTLChn'],
 'ToGrp': ['AbToGrp'],
 'CToSgrp': ['CIdToSgrp', 'CRToSgrp', 'CToMon'],
 'CToMon': ['CIdToMon', 'CIToMon', 'CRToMon'],
 'CIToMon': ['CIRToMon'],
 'CIdToSgrp': ['CIdRToSgrp', 'CIdToMon'],
 'CIdToMon': ['CIdRToMon'],
 'CToDivLat': ['CRToMag'],
 'CRToMag': ['CInToMag', 'CRToSgrp'],
 'CRToSgrp': ['CInToSgrp', 'CIdRToSgrp', 'CRToMon'],
 'CRToMon': ['CIdRToMon', 'CIRToMon', 'CInToMon'],
 'CIRToMon': ['IMTLChn'],
 'CIdRToSgrp': ['CIdInToSgrp', 'CIdRToMon'],
 'CIdRToMon': ['CIdInToMon'],
 'CInToMag': ['CInToSgrp'],
 'CInToSgrp': ['CIdInToSgrp', 'CInToMon'],
 'CInToMon': ['CIdInToMon', 'IMTLChn', 'AbLGrp'],
 'IMTLChn': ['TrivA'],
 'AbToGrp': ['TrivA'],
 'ToRng': ['ToFld'],
 'CToRng': ['ToFld'],
 'ToFld': [],

 'BA': ['pBA', 'BMag', 'BImpA', 'BUn', 'BNUn'],
 'pBA': ['BMon'],
 'BUn': ['BRUn', 'MA', 'CA$_2$', 'BRMod'],
 'BNUn': ['BGalLat'],
 'MA': ['TA', 'MonA'],
 'TA': ['TrivA'],
 'MonA': ['TrivA'],
 'BMag': ['BSgrp', 'BLrMag'],
 'BSgrp': ['BCSgrp', 'BLrSgrp', 'BMon', 'BIdSgrp'],
 'BMon': ['BCMon', 'BIMon', 'BLrMon', 'BIdMon'],
 'BIMon': ['BCIMon', 'BILrMon'],
 'BIdSgrp': ['BCIdSgrp', 'BIdLrSgrp', 'BIdMon'],
 'BIdMon': ['BCIdMon', 'BIdLrMon'],
 'BImpA': ['BLrMag', 'BDivLat'],
 'BLrMag': ['BRMag', 'BLrSgrp'],
 'BLrSgrp': ['BRSgrp', 'BLrMon', 'BIdLrSgrp'],
 'BLrMon': ['BRL', 'BILrMon', 'BIdLrMon'],
 'BILrMon': ['BIRL'],
 'BIdLrSgrp': ['BIdRSgrp', 'BIdLrMon'],
 'BIdLrMon': ['BIdRL'],
 'BRUn': ['BInFL'],
 'BDivLat': ['BCDivLat', 'BRMag'],
 'BRMag': ['BCRMag', 'BInMag', 'BRSgrp', 'NA'],
 'BRSgrp': ['BCRSgrp', 'BInSgrp', 'BRL', 'BIdRSgrp'],
 'BRL': ['BCRL', 'BIRL', 'BInFL', 'BIdRL'],
 'BIRL': ['BCIRL', 'BIInFL', 'SeqA'],
 'BIdRSgrp': ['BCIdRSgrp', 'BIdInSgrp', 'BIdRL'],
 'BIdRL': ['BCIdRL', 'BIdInFL'],
 'BGalLat': ['BInLat'],
 'BInLat': ['BInMag'],
 'BInMag': ['BCyInMag', 'BInSgrp'],
 'BInSgrp': ['BCyInSgrp', 'BInFL', 'BIdInSgrp'],
 'BInFL': ['BCyInFL', 'BIInFL', 'BIdInFL'],
 'BIInFL': ['BCyIInFL'],
 'BCyInMag': ['BCInMag', 'BCyInSgrp'],
 'BCyInSgrp': ['BCInSgrp', 'BCyInFL', 'BCyIdInSgrp'],
 'BCyInFL': ['BCInFL', 'BCyIInFL', 'BCyIdInFL'],
 'BCyIInFL': ['BCIInFL'],
 'BCSgrp': ['BCRSgrp', 'BCMon', 'BCIdSgrp', 'BSlat'],
 'BCMon': ['BCIMon', 'BCRL', 'BCIdMon'],
 'BCIMon': ['BCIRL'],
 'BSlat': ['TrivA'],
 'BCIdSgrp': ['BCIdRSgrp', 'BCIdMon'],
 'BCIdMon': ['BCIdRL'],
 'BCDivLat': ['BCRMag'],
 'BCRMag': ['BCInMag', 'BCRSgrp'],
 'BCRSgrp': ['BCInSgrp', 'BCRL', 'BCIdRSgrp'],
 'BCRL': ['BCIRL', 'BCInFL', 'BCIdRL'],
 'BCIRL': ['BCIInFL'],
 'BCIdRSgrp': ['BCIdInSgrp', 'BCIdRL'],
 'BCIdRL': ['BCIdInFL'],
 'BCInMag': ['BCInSgrp'],
 'BCInSgrp': ['BCInFL', 'BCIdInSgrp'],
 'BCInFL': ['BCIInFL', 'BCIdInFL'],
 'BCIInFL': ['TrivA'],
 'CA$_2$': ['TrivA'],
 'SeqA': ['RA'],
 'NA': ['RA'],
 'RA': ['IRA'],
 'IRA': ['TrivA'],
 'BRMod': ['TrivA'],

 'Set': ['pSet', 'Unar', 'Mag'],
 'pSet': ['Mon'],
 'Unar': ['RUnar', 'Mset'],
 'AAlg': [],
 'BCI': [],
 'RtQgrp': ['RtLp', 'Qgrp'],
 'Qgrp': ['Lp', 'CQgrp', 'InMag', 'MouQgrp'],
 'RtLp': ['Lp'],
 'MouQgrp': ['MouLp'],
 'Lp': ['MouLp', 'LNeofld'],
 'MouLp': ['Grp'],
 'Shell': ['Srng$_{01}$'],
 'Mag': ['CnjMag', 'Sgrp', 'LtQgrp', 'DivA', 'OrdA', 'QtMag', 'MedMag', 'Qnd', 'BCI', 'RtQgrp', 'Shell', 'Dtoid'],
 'Bnd': ['IdMon', 'NBnd', 'RzBnd'],
 'NBnd': ['RecBnd'],
 'RecBnd': ['Slat'],
 'Sgrp': ['Bnd', 'LtCanSgrp', 'CSgrp', 'Mon', 'LrSgrp', 'Sgrp$_0$', 'RegSgrp'],
 'Sgrp$_0$': ['Srng$_0$'],
 'RegSgrp': ['Bnd', 'InvSgrp'],
 'InvSgrp': ['CliffSgrp', 'CInvSgrp'],
 'Mon': ['LtCanMon', 'CMon', 'IdMon'],
 'CanSgrp': ['CanMon', 'CanCSgrp'],
 'CanMon': ['CanCMon', 'Grp'],
 'Grp': ['PGrp', 'NlGrp', 'AbGrp', 'pGrp', 'NRng'],
 'AbpGrp': ['BGrp'],
 'CMag': ['CSgrp'],
 'CSgrp': ['Slat', 'CMon', 'CanCSgrp', 'qMV'],
 'LtCanSgrp': ['LtCanMon', 'CanSgrp'],
 'LtCanMon': ['CanMon'],
 'CanCSgrp': ['CanCMon'],
 'CInvSgrp': ['AbGrp', 'Slat'],
 'CMon': ['CanCMon', 'Slat$_1$', 'Srng'],
 'CanCMon': ['AbGrp'],
 'AbGrp': ['BGrp', 'pAbGrp', 'Rng'],
 'BGrp': ['TrivA'],
 'NFld': ['Fld'],
 'NRng': ['Rng', 'NRng$_1$'],
 'NRng$_1$': ['Rng$_1$'],
 'Neofld': ['DivRng'],
 'Srng': ['IdSrng', 'Srng$_1$', 'Srng$_0$', 'CSrng'],
 'Srng$_1$': ['Srng$_{01}$', 'Sfld'],
 'Srng$_0$': ['IdSrng$_0$'],
 'Srng$_{01}$': ['IdSrng$_{01}$', 'Rng$_1$'],
 'Rng': ['CRng', 'Rng$_1$'],
 'Rng$_1$': ['CRng$_1$', 'OreDom', 'NFld', 'RegRng'],
 'RegRng': ['DivRng', 'CRegRng'],
 'CRegRng': ['Fld'],
 'CSrng': ['CSrng$_1$', 'CSrng$_0$'],
 'CSrng$_1$': ['CSrng$_{01}$'],
 'CSrng$_0$': ['CSrng$_{01}$'],
 'CSrng$_{01}$': [],
 'CRng': ['CRng$_1$', 'Fld'],
 'CRng$_1$': ['BA', 'IntDom'],
 'IntDom': ['UFDom'],
 'DivRng': ['Fld'],
 'Sfld': ['Fld'],
 'Fld': [],
 'CnjMag': ['CMag'],
 'Dtoid': ['Slat'],
 'UFDom': ['PIDom'],
 'OreDom': ['DivRng'],
 'PIDom': ['EucDom'],
 'EucDom': ['Fld'],
 'Mset': ['Gset'],
 'Gset': ['RMod'],
 'RMod': ['FVec'],
 'FVec': ['NaA'],
 'JorA': [],
 'LNeofld': ['Neofld'],
 'BilinA': ['LieA', 'AAlg'],
 'CliffSgrp': ['Grp'],
 'LieA': [],
 'MedMag': [],
 'NlGrp': ['AbGrp'],
 'NaA': ['BilinA', 'JorA'],
 'OrdA': ['Bnd'],
 'pGrp': ['AbpGrp', 'CypGrp'],
 'Qnd': [],
 'qMV': ['MV','sqMV'],
 'sqMV': [],
 'QtMag': [],
 'TrivA': []}

In [151]:
p=allclasses(chapter(2))
j=allclasses(chapter(3))
m=allclasses(chapter(4))
l=allclasses(chapter(5))
d=allclasses(chapter(6))
t=allclasses(chapter(7))
b=allclasses(chapter(8))
u=allclasses(chapter(9))

Number of classes: 56
Number of classes: 44
Number of classes: 42
Number of classes: 83
Number of classes: 91
Number of classes: 56
Number of classes: 61
Number of classes: 80


In [153]:
jl=['Lat',
 'pLat',
 'lbLat',
 'ubLat',
 'LUn',
 'LNUn',
 'GalLat',
 'LMag',
 'LSgrp',
 'LMon',
 'IdSrng$_0$',
 'IdSrng$_{01}$',
 'KA',
 'ILMon',
 'IdLSgrp',
 'IdLMon',
 'LImpA',
 'LrLMag',
 'LrLSgrp',
 'LrLMon',
 'ILrLMon',
 'IdLrLSgrp',
 'IdLrLMon',
 'RLUn',
 'DivLat',
 'RLMag',
 'RLSgrp',
 'RLMon',
 'IRLMon',
 'IdRLSgrp',
 'IdRLMon',
 'CLSgrp',
 'CLMon',
 'CILMon',
 'CIdLSgrp',
 'CIdLMon',
 'CDivLat',
 'BCKL',
 'CRLMag',
 'CRLSgrp',
 'CRLMon',
 'CIRLMon',
 'CIdRLSgrp',
 'CIdRLMon']

ml=['Lat',
 'pLat',
 'LUn',
 'LNUn',
 'GalLat',
 'LMag',
 'LSgrp',
 'LMon',
 'ILMon',
 'IdLSgrp',
 'IdLMon',
 'LImpA',
 'LrLMag',
 'LrLSgrp',
 'LrLMon',
 'ILrLMon',
 'RtHp',
 'IdLrLSgrp',
 'IdLrLMon',
 'RLUn',
 'DivLat',
 'RLMag',
 'RLSgrp',
 'RLMon',
 'IRLMon',
 'IdRLSgrp',
 'IdRLMon',
 'CLSgrp',
 'CLMon',
 'CILMon',
 'CIdLSgrp',
 'CIdLMon',
 'CDivLat',
 'BCKLat',
 'CRLMag',
 'CRLSgrp',
 'CRLMon',
 'CIRLMon',
 'Hp',
 'CIdRLSgrp',
 'CIdRLMon',
 'BrSlat']

ld=['DLat',
 'pDLat',
 'bDLat',
 'DLUn',
 'DLNUn',
 'DLMag',
 'DLSgrp',
 'DLMon',
 '',
 '',
 '',
 '',
 'DILMon',
 'DIdLSgrp',
 'DIdLMon',
 'DLImpA',
 'DLrLMag',
 'DLrLSgrp',
 'DLrLMon',
 'DILrLMon',
 'DIdLrLSgrp',
 'DIdLrLMon',
 'DRLUn',
 'DDivLat',
 'DRLMag',
 'DRLSgrp',
 'DRL',
 'bDRLat',
 'DIRL',
 'DIdRLSgrp',
 'DIdRL',
 'DFL',
 'DFL$_c$',
 'DFL$_e$',
 'DFL$_w$',
 'DFL$_{ec}$',
 'DFL$_{ew}$',
 'DGalLat',
 'DInLat',
 'DInLMag',
 'DInLSgrp',
 'DInFL',
 'DIInFL',
 'CyDInLMag',
 'CyDInLSgrp',
 'CyDInFL',
 'CyDIInFL',
 'CDLSgrp',
 'CDLMon',
 'CDILMon',
 'CDIdLSgrp',
 'CDIdLMon',
 'CDDivLat',
 '',
 'CDRLMag',
 'CDRLSgrp',
 'CDRL',
 'CDIRL',
 'CDIdRLSgrp',
 'CDIdRL',
 'CDIdInFL',
 'CDInLMag',
 'CDInLSgrp',
 'CDInFL',
 'CDIInFL',
 '',
 '',
 '',
 '',
 '',
 '',
 'CDdLat',
 '',
 '',
 '',
 'DBilat',
 'CanDRL',
 '',
 '',
 '',
 '',
 '',
 '']

dt=['ToLat',
 'pToLat',
 'bToLat',
 'ToUn',
 'ToNUn',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'ToMag',
 'ToSgrp',
 'ToMon',
 'IToMon',
 'IdToSgrp',
 'IdToMon',
 'ToImpA',
 'LrToMag',
 'LrToSgrp',
 'LrToMon',
 'ILrToMon',
 'IdLrToSgrp',
 'IdLrToMon',
 'RToUn',
 'ToDivLat',
 'RToMag',
 'RToSgrp',
 'RToMon',
 'IRToMon',
 'IdRToSgrp',
 'IdRToMon',
 'GalToLat',
 'InToLat',
 'InToMag',
 'InToSgrp',
 '',
 '',
 'CyInToMag',
 'CyInToSgrp',
 '',
 '',
 'ToGrp',
 '',
 'CToSgrp',
 'CToMon',
 'CIToMon',
 'CIdToSgrp',
 'CIdToMon',
 'CivToLat',
 'CRToMag',
 '',
 '',
 '',
 '',
 '',
 'CInToMag',
 '',
 '',
 '',
 'AbToGrp',
 'GBLChn',
 'GMVChn',
 'ToWHp',
 'BrA',
 '',
 '',
 'CRToSgrp',
 'CRToMon',
 'CIRToMon',
 'MTLChn',
 'BLChn',
 'MVChn',
 '',
 'GödChn',
 'CIdRToSgrp',
 'CIdRToMon',
 'CInToSgrp',
 'CInToMon',
 'DunnMon',
 'CIInRToMon',
 '',
 '',
 '',
 '',
 '',
 'ToRng',
 'CToRng',
   '','']
 
db=['BLat', 'pBLat', 'BBLat', 'BUn', 'BNUn', '', '', '', '',
 '', '', '', '', '', 'BMag', 'BSgrp', 'BMon', 'BIMon',
 'BIdSgrp', 'BIdMon', 'BImpA', 'BLrMag', 'BLrSgrp', 'BILrMon',
 'BIdLrSgrp', 'BIdLrMon', 'BRUn', 'BDivLat', 'BRMag', 'BRSgrp',
 'BRL', 'BIRL', 'BIdRSgrp', 'BIdRL', 'BGalA', 'BInLat',
 'BInMag', 'BInSgrp', 'BInFL', 'BIInFL', 'BCyInMag',
 'BCyInSgrp',
 'BCyInFL',
 'BCyIInFL',
 '',
 '',
 'BCSgrp',
 'BCMon',
 'BCIMon',
 'BCIdSgrp',
 'BCIdMon',
 'BCDivLat',
 'BCRMag',
 '',
 '',
 '',
 '',
 '',
 'BCInMag',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'BCRSgrp',
 'BCRMon',
 'BCIRMon',
 '',
 '',
 '',
 '',
 '',
 'BCIdRSgrp',
 'BCIdRMon',
 'BCInSgrp',
 'BCInMon',
 '',
 'BCIInRMon',
 '',
 '',
 '',
 '',
 '',
 '',
 '','','']

In [154]:
pj=['Jslat',
 'pJslat',
 'JUn',
 'JNUn',
 'JMag',
 'JSgrp',
 'JMon',
 'IJMon',
 'IdJSgrp',
 'IdJMon',
 'JImpA',
 'LrJMag',
 'LrJSgrp',
 'LrJMon',
 'ILrJMon',
 'IdLrJSgrp',
 'IdLrJMon',
 'RJUn',
 'DivJslat',
 'RJMag',
 'RJSgrp',
 'RJMon',
 'IRJMon',
 'IdRJSgrp',
 'IdRJMon',
 'GalJslat',
 'InLat',
 'InLMag',
 'InLSgrp',
 'InLMon',
 'IInLMon',
 'CyInLMag',
 'CyInLSgrp',
 'CyInLMon',
 'CyIInLMon',
 'LGrp',
 'CJSgrp',
 'CJMon',
 'CIJMon',
 'CIdJSgrp',
 'CIdJMon',
 'CDivJslat',
 'BCKJ',
 'CRJMag',
 'HilAJ',
 'TarAJ',
 'CRJSgrp',
 'CRJMon',
 'CIRJMon',
 'CIdRJSgrp',
 'CIdRJMon',
 'CInLMag',
 'CInLSgrp',
 'CInLMon',
 'CIInLMon',
 'AbLGrp']
pm=['Mslat',
 'pMslat',
 'MUn',
 'MNUn',
 'MMag',
 'MSgrp',
 'MMon',
 'IMMon',
 'IdMSgrp',
 'IdMMon',
 'MImpA',
 'LrMMag',
 'LrMSgrp',
 'LrMMon',
 'ILrMMon',
 'IdLrMSgrp',
 'IdLrMMon',
 'RMUn',
 'DivMslat',
 'RMMag',
 'RMSgrp',
 'RMMon',
 'IRMMon',
 'IdRMSgrp',
 'IdRMMon',
 'GalMslat',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'CMSgrp',
 'CMMon',
 'CIMMon',
 'CIdMSgrp',
 'CIdMMon',
 'CDivMslat',
 'BCKM',
 'CRMMag',
 'HilAM',
 'TarAM',
 'CRMSgrp',
 'CRMMon',
 'CIRMMon',
 'CIdRMSgrp',
 'CIdRMMon',
 '',
 '',
 '',
 '',
 '']
pp={p[i]:([pj[i],pm[i]] if pm[i]!="" else [pj[i]]) for i in range(len(p))}
jj={j[i]:[jl[i]] for i in range(len(j))}
mm={m[i]:[ml[i]] for i in range(len(m))}
ll={l[i]:[ld[i]] for i in range(len(l))}
dd={d[i]:([dt[i],db[i]] if db[i]!="" else [dt[i]]) for i in range(len(d))}
len(pp),len(jj),len(mm),len(ll),len(dd),

(56, 44, 42, 83, 91)

8

In [155]:
cpo=allclassposets()
print(len(cpo))
tkz="".join(cpo)
nli=allnodes(tkz)
print(len(nli),len(set(nli)))
nds = {x:lowercovers(x,tkz) for x in nli}
for x in pp.keys():
    nds[x]=nds[x]+pp[x] if x in nds.keys() else pp[x]
for x in jj.keys():
    nds[x]=nds[x]+jj[x] if x in nds.keys() else jj[x]
for x in mm.keys():
    nds[x]=nds[x]+mm[x] if x in nds.keys() else mm[x]
for x in ll.keys():
    nds[x]=nds[x]+ll[x] if x in nds.keys() else ll[x]
for x in dd.keys():
    nds[x]=nds[x]+dd[x] if x in nds.keys() else dd[x]
len(nds)

8
Number of nodes: 450
450 448


514

In [156]:
lowercovers('JMon',tkz)

['IdJMon', 'IJMon', 'IdSrng$_{01}$', 'CJMon', 'LrJMon']

In [157]:
nds['JMon']

['IdJMon', 'IJMon', 'IdSrng$_{01}$', 'CJMon', 'LrJMon', 'LMon']

In [159]:
scp=subclassposet()
len(scp)

513

In [160]:
for x in scp.keys():
    scp[x]+=([y for y in nds[x] if y!='' and y not in scp[x]] if x in nds.keys() else [])

In [89]:
d=set(s.replace('$\\ell$','L') for s in cli)-set(nli)
len(d)

0

In [173]:
sc={"a":["b"],"b":[]}
from graphviz import Graph
from IPython.display import display_html
P = Graph()
P.attr('node', shape='circle', width='.15', height='.15', fixedsize='true', fontsize='10')
#P.attr('node', shape='oval', fontsize='10')
for x in sc: P.node(str(x), color='red')
for x in sc: P.edges([(str(x),str(y)) for y in sc[x]])
display_html(P._repr_svg_(),raw=True)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: %3 Pages: 1 -->
 
 
 %3 
 
<!-- a -->
 
 a 
 
 a 
 
<!-- b -->
 
 b 
 
 b 
 
<!-- a--b -->
 
 a--b

In [162]:
di={0:"a",1:"b"}
for x in di: print(x)

0
1


In [85]:
d=set(nli)-set(s.replace('$\\ell$','L') for s in cli)
print(len(d))
d

53


{'AbGrpe',
 'BCIdInFL',
 'BCIdInSgrp',
 'BCyIdInFL',
 'BCyIdInSgrp',
 'BIdInFL',
 'BIdInSgrp',
 'CDIdInFL',
 'CDIdInLSgrp',
 'CDivA',
 'CIInSlMon',
 'CIdInLSgrp',
 'CIdInPoMon',
 'CIdInPoSgrp',
 'CIdInSlMon',
 'CIdInSlSgrp',
 'CIdInToMon',
 'CIdInToSgrp',
 'CInMag',
 'CInSgrp',
 'CQgrp',
 'CyDIdInFL',
 'CyDIdInLSgrp',
 'CyIdInFL',
 'CyIdInLSgrp',
 'CyIdInPoMon',
 'CyIdInPoSgrp',
 'CyIdInToMon',
 'CyIdInToSgrp',
 'CyInMag',
 'CyInSgrp',
 'CypGrp',
 'DIdInFL',
 'DIdInLSgrp',
 'DivA',
 'Grpe',
 'IdInFL',
 'IdInLSgrp',
 'IdInPoMon',
 'IdInPoSgrp',
 'IdInToMon',
 'IdInToSgrp',
 'IdMon',
 'InMag',
 'InSgrp',
 'LrSgrp',
 'LtQgrp',
 'RUnar',
 'RzBnd',
 'Slat',
 'Slat1',
 'pAbGrp',
 'pSet'}

In [33]:
%%time
checkfs([1])#,5,6,7,8])

[0, 56, 0, 0, 0, 0, 0, 0, 0, 0]
1 ['x<=y->x=y', 'x<=x', 'x<=y & y<=x -> x=y', 'x<=y & y<=z -> x<=z']
Pos
['(x<=x)', '(((x<=y) & (y<=z)) -> (x<=z))', '(((x<=y) & (y<=x)) -> (x=y))']


Number of nonisomorphic models of cardinality 2  is  1


Number of nonisomorphic models of cardinality 3  is  1


Number of nonisomorphic models of cardinality 4  is  1


Number of nonisomorphic models of cardinality 5  is  1


...

Number of nonisomorphic models of cardinality 2  is  1


Number of nonisomorphic models of cardinality 3  is  1


Number of nonisomorphic models of cardinality 4  is  2


Number of nonisomorphic models of cardinality 5  is  1


Number of nonisomorphic models of cardinality 6  is  1
AbPoGrp [1, 1, 1, 2, 1, 1, 1, 3, 2, 1]
CPU times: user 1.94 s, sys: 31.2 s, total: 33.1 s
Wall time: 1min 8s


In [23]:
writefile("ch12.tex",latex_st)

'Wrote 909031 characters'

In [212]:
print(finitememberslatex(fs))

$f_1 = 1$,
$f_2 = 2$,
$f_3 = 9$,
$f_4 = 63$,
$f_5 = 492$



In [208]:
%%time
SkLat=['(((x^y)^z)=(x^(y^z)))', '((x^x)=x)', '(((x v y) v z)=(x v (y v z)))', '((x v x)=x)', '((x^(x v y))=x)', '((x v (x^y))=x)', '(((x v y)^y)=y)', '(((x^y) v y)=y)',
      'x<=y<->x^y=x&y^x=x']
#a=p9(SkLat,[],1000,1000,[4])
a=p9(SkLat,["x<=y->x^z<=y^z"],100,1000)#,[4])

CPU times: user 2.48 ms, sys: 48.9 ms, total: 51.3 ms
Wall time: 99.5 ms


In [209]:
a

[
 Model(cardinality = 3, index = 0,
 operations = {
 "^":[
 [0,0,0],
 [0,1,2],
 [2,2,2]], 
 "v":[
 [0,1,2],
 [1,1,1],
 [0,1,2]], "c1":0, "c2":1, "c3":2},
 relations = {
 "<=":[
 [1,1,0],
 [0,1,0],
 [0,1,1]]})]

In [274]:
%%time
fs=finespectrum(lat,"CIRL",5,True)

CIRL
['((x<=y) -> ((x*z)<=(y*z)))', '((x<=y) -> ((z*x)<=(z*y)))', '(((x*y)*z)=(x*(y*z)))', '((x*1)=x)', '((1*x)=x)', '(x<=1)', '(((x*y)<=z) <-> (y<=(x\\ z)))', '(((x*y)<=z) <-> (x<=(z/y)))', '((x*y)=(y*x))']
Number of nonisomorphic models of cardinality 2  is  1


Number of nonisomorphic models of cardinality 3  is  2
Number of nonisomorphic models of cardinality 4  is  7


Number of nonisomorphic models of cardinality 5  is  26
CPU times: user 179 ms, sys: 227 ms, total: 406 ms
Wall time: 980 ms


In [196]:
%%time
#RtHp=msl+['((x<=y) <-> ((y/x)=1))', '(((x*y)*z)=(x*(y*z)))', '((x*1)=x)', '((1*x)=x)', '((x/(y*z))=((x/z)/y))', '((x/x)=1)', '(((x/y)*y)=((y/x)*x))']+['x<=y -> z*x<=z*y']
RtHp=msl+['((x<=y) <-> ((y/x)=1))', '(((x*y)*z)=(x*(y*z)))', '((x*1)=x)', '((1*x)=x)', '((x/(y*z))=((x/z)/y))', '((x/x)=1)', '(((x/y)*y)=((y/x)*x))',
          '((y*z)\ x)=(z\ (y\ x))', '((x\ x)=1)', '((x*(x\ y))=(y*(y\ x)))','((x<=y) <-> ((x\ y)=1))']
         #]+['x<=y -> z/y<=z/x']
#a=p9(RtHp,['x<=y -> z/y<=z/x'],0,1000) #provable
a=p9(RtHp,[],1000,1000,[5])#[1, 1, 2, 6, 17] GHp:[1, 1, 2, 5, 10]
#a=p9(RtHp,['x<=y -> z*x<=z*y'],0,1000)

Number of nonisomorphic models of cardinality 2  is  1
Number of nonisomorphic models of cardinality 3  is  2


Number of nonisomorphic models of cardinality 4  is  5


Number of nonisomorphic models of cardinality 5  is  10
Fine spectrum:  [1, 1, 2, 5, 10]
CPU times: user 25.9 ms, sys: 368 ms, total: 394 ms
Wall time: 876 ms


In [194]:
a

[
 Proof(syntax="Prover9", formula_list=[
 [1, 'x ^ y = x <-> x <= y # label(non_clause)', []],
 [2, 'x <= y <-> y / x = 1 # label(non_clause)', []],
 [3, 'x <= y -> z / y <= z / x # label(non_clause)', []],
 [4, 'x <= y -> z * x <= z * y # label(non_clause) # label(goal)', []],
 [5, 'x ^ y ^ z = x ^ (y ^ z)', []],
 [6, 'x ^ y = y ^ x', []],
 [7, 'x ^ x = x', []],
 [8, 'x ^ y != x | x <= y', [1]],
 [9, 'x ^ y = x | -(x <= y)', [1]],
 [10, '-(x <= y) | y / x = 1', [2]],
 [11, 'x <= y | y / x != 1', [2]],
 [12, 'x / (y * z) = x / z / y', []],
 [13, 'x / y / z = x / (z * y)', [12]],
 [14, 'x / x = 1', []],
 [15, '-(x <= y) | z / y <= z / x', [3]],
 [16, 'c1 <= c2', [4]],
 [17, '-(c3 * c1 <= c3 * c2)', [4]],
 [18, 'x ^ y ^ z = y ^ (x ^ z)', [6, 5]],
 [19, 'x ^ (y ^ z) = y ^ (x ^ z)', [5, 18]],
 [20, 'x ^ y = x ^ (x ^ y)', [7, 5]],
 [21, 'x ^ (x ^ y) = x ^ y', [20]],
 [22, 'x ^ y != y | y <= x', [6, 8]],
 [23, 'x <= y / z | y / (x * z) != 1', [13, 11]],
 [24, 'x / c2 <= x / c1', [15, 16]],


In [102]:
%%time
IdInPoMon=po+['x*x=x','(~(-(x))=x)', '(-(~(x))=x)', '(((x*y)<=z) <-> (y<=~((-(z)*x))))', '(((x*y)<=z) <-> (x<=-((y*~(z)))))', '(((x*y)*z)=(x*(y*z)))', '((x*1)=x)', '((1*x)=x)']
a=p9(IdInPoMon,[],1000,1000,[9])

Number of nonisomorphic models of cardinality 2  is  1
Number of nonisomorphic models of cardinality 3  is  1


Number of nonisomorphic models of cardinality 4  is  2
Number of nonisomorphic models of cardinality 5  is  2


Number of nonisomorphic models of cardinality 6  is  4


Number of nonisomorphic models of cardinality 7  is  4


Number of nonisomorphic models of cardinality 8  is  9


Number of nonisomorphic models of cardinality 9  is  10
Fine spectrum:  [1, 1, 1, 2, 2, 4, 4, 9, 10]
CPU times: user 83.6 ms, sys: 625 ms, total: 709 ms
Wall time: 4.3 s


In [103]:
%%time
CIdInPoMon=po+['x*y=y*x','x*x=x','(~(-(x))=x)', '(-(~(x))=x)', '(((x*y)<=z) <-> (y<=~((-(z)*x))))', '(((x*y)<=z) <-> (x<=-((y*~(z)))))', '(((x*y)*z)=(x*(y*z)))', '((x*1)=x)', '((1*x)=x)']
a=p9(CIdInPoMon,[],1000,1000,[9])

Number of nonisomorphic models of cardinality 2  is  1
Number of nonisomorphic models of cardinality 3  is  1


Number of nonisomorphic models of cardinality 4  is  2
Number of nonisomorphic models of cardinality 5  is  2


Number of nonisomorphic models of cardinality 6  is  4


Number of nonisomorphic models of cardinality 7  is  4


Number of nonisomorphic models of cardinality 8  is  9


Number of nonisomorphic models of cardinality 9  is  10
Fine spectrum:  [1, 1, 1, 2, 2, 4, 4, 9, 10]
CPU times: user 89 ms, sys: 654 ms, total: 743 ms
Wall time: 3.52 s


In [36]:
%%time
fs=finespectrum(axioms[1],"CInPoMon",9,True) # A328705!!!!!

Number of nonisomorphic models of cardinality 2  is  2


Number of nonisomorphic models of cardinality 3  is  2


Number of nonisomorphic models of cardinality 4  is  5


Number of nonisomorphic models of cardinality 5  is  2


Number of nonisomorphic models of cardinality 6  is  4


Number of nonisomorphic models of cardinality 7  is  2


Number of nonisomorphic models of cardinality 8  is  10


Number of nonisomorphic models of cardinality 9  is  5
CPU times: user 37.5 ms, sys: 1.47 s, total: 1.51 s
Wall time: 6.94 s


In [39]:
finespectrum(["c=c"],"AbGrp",8)

Number of nonisomorphic models of cardinality 2  is  2


Number of nonisomorphic models of cardinality 3  is  2


Number of nonisomorphic models of cardinality 4  is  5


Number of nonisomorphic models of cardinality 5  is  2


Number of nonisomorphic models of cardinality 6  is  4


Number of nonisomorphic models of cardinality 7  is  2


Number of nonisomorphic models of cardinality 8  is  10


[1, 2, 2, 5, 2, 4, 2, 10]

In [40]:
finespectrum(["c=c"],"Grp",8)

Number of nonisomorphic models of cardinality 2  is  2


Number of nonisomorphic models of cardinality 3  is  2


Number of nonisomorphic models of cardinality 4  is  5


Number of nonisomorphic models of cardinality 5  is  2


Number of nonisomorphic models of cardinality 6  is  7


Number of nonisomorphic models of cardinality 7  is  2


Number of nonisomorphic models of cardinality 8  is  17


[1, 2, 2, 5, 2, 7, 2, 17]

In [158]:
m=re.findall(r"\\hypertarget{(.*?)}{",latex_st,flags=re.DOTALL)
len(m)

501

In [95]:
len(Po)

280

In [78]:
def rep(m): return "\\hypertarget{"+m.group(1)+"}{\\section{"+m.group(1)+": "+m.group(2)+"}}"
st1=re.sub(r"\\hypertarget{(.*?)}{\\section{(.*?)}}",rep,latex_st,flags=re.DOTALL)

In [81]:
len(st1)

121924

In [80]:
st1=st1.replace(r"\abbreviation{",r"%\abbreviation{")

In [92]:
ch=["" for n in range(4)]

In [96]:
n=1
#st1=chapter(n)
st1=latex_st
len(st1)

604920

In [97]:
m=re.findall(r"\\hypertarget{(.*?)}{",st1,flags=re.DOTALL)
len(m)

338

In [16]:
m

['Lat',
 'pLat',
 '$\\ell$Mag',
 '$\\ell$Sgrp',
 'C$\\ell$Sgrp',
 '$\\ell$Mon',
 'C$\\ell$Mon',
 'I$\\ell$Mon',
 'CI$\\ell$Mon',
 'Id$\\ell$Sgrp',
 'CId$\\ell$Sgrp',
 'Id$\\ell$Mon',
 'CId$\\ell$Mon',
 '$\\ell$ImpA',
 'Lr$\\ell$Mag',
 'Lr$\\ell$Sgrp',
 'Lr$\\ell$Mon',
 'ILr$\\ell$Mon',
 'IdLr$\\ell$Sgrp',
 'IdLr$\\ell$Mon',
 'DivLat',
 'CDivLat',
 'R$\\ell$Mag',
 'CR$\\ell$Mag',
 'R$\\ell$Sgrp',
 'CR$\\ell$Sgrp',
 'RL',
 'CRL',
 'IRL',
 'CIRL',
 'IdR$\\ell$Sgrp',
 'CIdR$\\ell$Sgrp',
 'IdRL',
 'CIdRL',
 '$\\ell$Un',
 '$\\ell$NUn',
 'GalLat',
 'InLat',
 'In$\\ell$Mag',
 'CyIn$\\ell$Mag',
 'CIn$\\ell$Mag',
 'In$\\ell$Sgrp',
 'CyIn$\\ell$Sgrp',
 'CIn$\\ell$Sgrp',
 'InFL',
 'CyInFL',
 'CInFL',
 'IInFL',
 'CyIInFL',
 'CIInFL',
 'ADLat',
 'BCKlat',
 'Bilat',
 'bLat',
 'bRLat',
 'CanRL',
 'CIdInFL',
 'CdLat',
 'CdMLat',
 'FL',
 'FL$_c$',
 'FL$_e$',
 'FL$_{ec}$',
 'FL$_{ew}$',
 'FL$_w$',
 'FRng',
 'ILLA',
 'JsdLat',
 'KLat',
 'LLA',
 'MsdLat',
 'MLat',
 'MOLat',
 'MALLA',
 'MultLat',
 'NdLat',


In [98]:
se = {cl:section_mo(cl,st1) for cl in m}

In [99]:
len(se)
[cl for cl in m if se[cl]==None]

[]

In [105]:
len(se)

336

In [107]:
len(set(m)) #-set(se.keys())

336

In [108]:
set(Po)-set(m)

set()

In [100]:
st2 = "".join(se[cl].group(1) for cl in Po)

In [101]:
len(st2)

498489

In [103]:
ch = [st1[:se[m[0]].start(1)]+st2+st1[se[m[-1]].end(1):]]
len(ch[0])

500002

In [7]:
len(Po)

58

In [9]:
set(Po)-set(m),set(m)-set(Po)

(set(), set())

In [3]:
print(st1[:2000])

\chapter{Partially ordered structures}

\hypertarget{Pos}{\section{Pos: Partially ordered sets}}

%\abbreviation{Pos}

\begin{definition}
A \emph{partially ordered set} (also called \emph{ordered set} or \emph{poset} for short) is a po-algebra $\mathbf{P}=\langle P,\le \rangle $
such that $P$ is a set and $\le $ is a binary relation on $P$ that is
\begin{itemize}
\item reflexive:  $x\le x$

\item transitive:  $x\le y$, $y\le z\implies x\le z$

\item antiymmetric:  $x\le y$, $y\le x\implies x=y$.
\end{itemize}
\end{definition}

\begin{definition}
A \emph{strict partial order} is a po-algebra $\langle P,<\rangle $
such that $P$ is a set and $<$ is a binary relation on $P$ that is

irreflexive:  $\neg(x<x)$

transitive:  $x<y$, $y<z\implies x<y$

Remark: 
The above definitions are related via: $x\le y\Longleftrightarrow x<y \mbox{or} x=y$ and 
$x<y\Longleftrightarrow x\le y$, $x\neq y$.

For a partially ordered set $\mathbf{P}$, define the dual $\mathbf{P}^{\partial }=\langle P,\geq \rang

In [0]:
for cl in m:
    latex_st = set_classname(cl,m+": "+classname(m),latex_st)

In [3]:
lc=subclassposet()

In [4]:
uc=superclassposet()

In [5]:
for c in lc.keys():
    for s in lc[c]:
        if s in uc.keys() and c not in uc[s]: print(c,s)

Pos Jslat
pPos pJslat
PoMag JMag
PoSgrp JSgrp
CPoSgrp CRPoSgrp
CPoSgrp CJSgrp
PoMon JMon
PoMon MMon
CPoMon CIdPoMon
CPoMon CRPoMon
CPoMon CJMon
CPoMon CMMon
IPoMon IJMon
IPoMon IMMon
CIPoMon Pocrim
CIPoMon CIJMon
CIPoMon CIMMon
IdPoSgrp IdLrPoSgrp
IdPoSgrp IdJSgrp
IdPoSgrp IdMSgrp
CIdPoSgrp CIdJSgrp
CIdPoSgrp CIdMSgrp
IdPoMon IdLrPoMon
IdPoMon IdJMon
IdPoMon IdMMon
CIdPoMon CIdRPoMon
CIdPoMon CIdJMon
CIdPoMon CIdMMon
PoImpA JImpA
PoImpA MImpA
LrPoMag LrJMag
LrPoMag LrMMag
LrPoSgrp RPoMon
Porim InPorim
InPoMon PoGrp
pJslat JMon
CJSgrp CRJSgrp
CJMon CRJMon
CIJMon CIRJMon
CIdJSgrp CIdJMon
IdJMon IdLrJMon
CIdJMon CIdRJMon
LrJSgrp RJMon
RJMon IdRJMon
CRJMon CIdRJMon
MSgrp $\ell$Sgrp
CMSgrp CR$\ell$Sgrp
CIRMMon CIRL
GalMslat GalLat
CI$\ell$Mon CIRL
CIRL CIInFL
CIdRL CIdInFL
MultLat $\ell$Sgrp
SkLat Lat


In [6]:
for c in uc.keys():
    for s in uc[c]:
        if s in lc.keys() and c not in lc[s]: print(c,s)

In [7]:
lc

{'Mslat': ['pMslat', 'MMag', 'MImpA', 'MUn', 'MNUn'],
 'pMslat': ['MMon'],
 'MMag': ['MSgrp', 'LrMMag'],
 'MSgrp': ['CMSgrp', 'IdMSgrp', 'LrMSgrp', '$\\ell$Sgrp', 'MMon'],
 'CMSgrp': ['CMMon', 'CIdMSgrp', 'CRMSgrp', 'CR$\\ell$Sgrp'],
 'MMon': ['CMMon', 'IdMMon', 'IMMon', 'LrMMon'],
 'CMMon': ['CIMMon', 'CRMMon'],
 'IMMon': ['CIMMon', 'ILrMMon'],
 'CIMMon': ['CIRMMon'],
 'IdMSgrp': ['CIdMSgrp', 'IdLrMSgrp', 'IdMMon'],
 'CIdMSgrp': ['CIdMMon', 'CIdRMSgrp'],
 'IdMMon': ['CIdMMon', 'IdLrMMon'],
 'CIdMMon': ['CIdRMMon'],
 'MImpA': ['DivMslat', 'LrMMag'],
 'LrMMag': ['RMMag', 'LrMSgrp'],
 'LrMSgrp': ['IdLrMSgrp', 'LrMMon', 'RMSgrp'],
 'LrMMon': ['IdLrMMon', 'ILrMMon', 'RMMon'],
 'ILrMMon': ['IRMMon'],
 'IdLrMSgrp': ['IdRMSgrp', 'IdLrMMon'],
 'IdLrMMon': ['IdRMMon'],
 'DivMslat': ['CDivMslat', 'RMMag'],
 'CDivMslat': ['CRMMag'],
 'RMMag': ['CRMMag', 'RMSgrp'],
 'CRMMag': ['CRMSgrp', 'CInMMag'],
 'RMSgrp': ['CRMSgrp', 'IdRMSgrp', 'RMMon'],
 'CRMSgrp': ['CIdRMSgrp', 'CRMMon'],
 'RMMon': ['IdRMM

In [8]:
uc

{'Mslat': ['Pos'],
 'pMslat': ['pPos', 'Mslat'],
 'MMag': ['PoMag', 'Mslat'],
 'MSgrp': ['PoSgrp', 'MMag'],
 'CMSgrp': ['CPoSgrp', 'MSgrp'],
 'MMon': ['MSgrp', 'pMslat'],
 'CMMon': ['MMon', 'CMSgrp'],
 'IMMon': ['MMon'],
 'CIMMon': ['CMMon', 'IMMon'],
 'IdMSgrp': ['MSgrp'],
 'CIdMSgrp': ['IdMSgrp', 'CMSgrp'],
 'IdMMon': ['IdMSgrp', 'MMon'],
 'CIdMMon': ['CIdMSgrp', 'IdMMon'],
 'MImpA': ['Mslat'],
 'LrMMag': ['MMag', 'MImpA'],
 'LrMSgrp': ['MSgrp', 'LrMMag'],
 'LrMMon': ['MMon', 'LrMSgrp'],
 'ILrMMon': ['IMMon', 'LrMMon'],
 'IdLrMSgrp': ['LrMSgrp', 'IdMSgrp'],
 'IdLrMMon': ['IdMMon', 'LrMMon', 'IdLrMSgrp'],
 'DivMslat': ['MImpA'],
 'CDivMslat': ['DivMslat'],
 'RMMag': ['LrMMag', 'DivMslat'],
 'CRMMag': ['RMMag', 'CDivMslat'],
 'RMSgrp': ['LrMSgrp', 'RMMag'],
 'CRMSgrp': ['CMSgrp', 'RMSgrp', 'CRMMag'],
 'RMMon': ['LrMMon', 'RMSgrp'],
 'CRMMon': ['CMMon', 'RMMon', 'CRMSgrp'],
 'IRMMon': ['RMMon', 'ILrMMon'],
 'CIRMMon': ['CIMMon', 'CRMMon', 'IRMMon'],
 'IdRMSgrp': ['IdLrMSgrp', 'RMSgrp'],

In [4]:
mm=re.findall(r"\\abbreviation{(.*?)}",latex_st,flags=re.DOTALL)
len(mm)

42

In [24]:
m=re.findall(r"\\hypertarget{(.*?)}{",latex_st,flags=re.DOTALL)
len(m)

502

In [151]:
m[54:97]

['Jslat',
 'pJslat',
 'lbJslat',
 'ubJslat',
 'JUn',
 'JNUn',
 'GalJslat',
 'JMag',
 'JSgrp',
 'JMon',
 'IdSRng$_0$',
 'IdSRng$_{01}$',
 'KA',
 'IJMon',
 'IdJSgrp',
 'IdJMon',
 'JImpA',
 'LrJMag',
 'LrJSgrp',
 'LrJMon',
 'ILrJMon',
 'IdLrJSgrp',
 'IdLrJMon',
 'DivJslat',
 'RJMag',
 'RJSgrp',
 'RJMon',
 'IRJMon',
 'IdRJSgrp',
 'IdRJMon',
 'CJSgrp',
 'CJMon',
 'CIJMon',
 'CIdJSgrp',
 'CIdJMon',
 'CDivJslat',
 'BCKJslat',
 'CRJMag',
 'CRJSgrp',
 'CRJMon',
 'CIRJMon',
 'CIdRJSgrp',
 'CIdRJMon']

In [27]:
def finespectralatex(cli):
    li = list(reversed(sorted([(finitemembers(s)[:10],s) for s in cli])))
    st = r"\begin{tabular}{|l|l|l|}\hline\nName& Fine spectrum& OEIS\\\hline"
    return st+"&\\\\\n".join(x[1]+"& "+str(x[0])[1:-1] for x in li)+"\hline\n\end{tabular}\n"
st1=finespectralatex(m)
print(st1[:1000])
writefile("ch3.tex",st1)

\begin{tabular}{|l|l|l|}\hline\nName& Fine spectrum& OEIS\\\hlinePoMag& 1, 16, 4051&\\
PoImpA& 1, 16, 3981&\\
PoSgrp& 1, 11, 173, 4753, 198838, 13457454, 4207546916&\\
Mag& 1, 10, 3330, 178981952, 2483527537094825, 14325590003318891522275680&\\
Srng& 1, 10, 132, 2341&\\
DivPos& 1, 7, 352&\\
CDivPos& 1, 7, 122, 10149&\\
CPoSgrp& 1, 7, 83, 1468, 37248, 1337698, 71748346&\\
MedMag& 1, 7, 75&\\
IdPoSgrp& 1, 7, 69, 1035&\\
MMag& 1, 6, 280&\\
JImpA& 1, 6, 245&\\
MImpA& 1, 6, 220&\\
JMag& 1, 6, 220&\\
ToMag& 1, 6, 175&\\
ToImpA& 1, 6, 175&\\
MultLat& 1, 6, 175&\\
D$\ell$Mag& 1, 6, 175&\\
D$\ell$ImpA& 1, 6, 175&\\
$\ell$Mag& 1, 6, 175&\\
$\ell$ImpA& 1, 6, 175&\\
LrPoMag& 1, 6, 110&\\
MSgrp& 1, 6, 70, 1437&\\
JSgrp& 1, 6, 61, 866&\\
D$\ell$Sgrp& 1, 6, 44, 479&\\
$\ell$Sgrp& 1, 6, 44, 479&\\
ToSgrp& 1, 6, 44, 386&\\
PoUn& 1, 6, 43, 452&\\
PoNUn& 1, 6, 39, 386, 5203&\\
BMag& 1, 6, 0, 1176&\\
BSgrp& 1, 6, 0, 93, 0, 0, 0&\\
BImpA& 1, 6, 0&\\
LrPoSgrp& 1, 5, 28, 273, 3788&\\
Sgrp& 1, 5, 24, 188, 191

'Wrote 14693 characters'

In [67]:
[cl for cl in m if fulldefinition(cl)!="none"]

['AbGrp',
 'Bnd',
 'BCI',
 'Mag',
 'CanCMon',
 'CanCSgrp',
 'CanMon',
 'CanSgrp',
 'CMag',
 'CInvSgrp',
 'CMon',
 'CRng',
 'CRng$_1$',
 'CSgrp',
 'Dtoid',
 'DRng',
 'Fld',
 'Grp',
 'InvSgrp',
 'LCanSgrp',
 'Lp',
 'MedMag',
 'Mon',
 'MouLp',
 'MouQgrp',
 'NRng',
 'NRng$_1$',
 'NBnd',
 'OrdA',
 'Qnd',
 'qMV',
 'Qgrp',
 'QtMag',
 'RBnd',
 'RtQgrp',
 'Rng',
 'Rng$_1$',
 'Sgrp',
 'Sgrp$_0$',
 'SRng',
 'SRng$_1$',
 'SRng$_{01}$',
 'SRng$_0$',
 'Shells']

In [134]:
for s in m: print(s.ljust(11, ' '),finitemembers(s))#,info=True))

Pos         [1, 2, 5, 16, 63, 318, 2045, 16999, 183231, 2567284, 46749427, 1104891746, 33823827452, 1338193159771, 68275077901156, 4483130665195087]
pPos        [1, 3, 11, 47, 243]
PoUn        [1, 6, 43, 452]
PoNUn       [1, 6, 39, 386, 5203]
PoMag       [1, 16, 4051]
PoSgrp      [1, 11, 173, 4753, 198838, 13457454, 4207546916]
PoMon       [1, 4, 37, 549]
IPoMon      [1, 1, 2, 11, 102, 1609]
IdPoSgrp    [1, 7, 69, 1035]
IdPoMon     [1, 3, 23, 238, 3356]
PoImpA      [1, 16, 3981]
LrPoMag     [1, 6, 110]
LrPoSgrp    [1, 5, 28, 273, 3788]
LrPoMon     [1, 2, 6, 32, 234, 2493]
Polrim      [1, 1, 2, 11, 102, 1609]
IdLrPoSgrp  [1, 3, 12, 71, 524]
IdLrPoMon   [1, 1, 3, 12, 59, 350]
RPoUn       [1, 6, 43, 452]
DivPos      [1, 7, 352]
RPoMag      [1, 3, 28, 1200]
RPoSgrp     [1, 3, 16, 154, 2100]
RPoMon      [1, 2, 5, 28, 186]
Porim       [1, 1, 2, 9, 49, 365]
IdRPoSgrp   [1, 1, 4, 24, 169]
IdRPoMon    [1, 1, 2, 8, 32, 148]
GalPos      [1, 4, 15, 83, 539]
InPos       [1, 3, 5, 16, 30, 108]
InPoM

 [1, 1, 0, 5, 0, 0, 0]
BCyIInFL    [1, 1, 0, 1, 0, 0, 0, 1]
BCSgrp      [1, 4, 0]
BCMon       [1, 2, 0, 9]
BCIMon      [1, 1, 0, 1, 0, 0]
BSlat       [1, 1, 0, 5, 0, 0, 0]
BCIdSgrp    [1, 2, 0, 5]
BCIdMon     [1, 2, 0, 4, 0]
BCDivLat    [1, 4, 0]
BCRMag      [1, 2, 0, 36]
BCRSgrp     [1, 2, 0, 16, 0]
BCRL        [1, 1, 0, 5, 0]
BCIRL       [1, 1, 0, 1, 0, 0]
BCIdRSgrp   [1, 1, 0, 3, 0, 0]
BCIdRL      [1, 1, 0, 2, 0, 0]
BCInMag     [1, 2, 0, 20, 0]
BCInSgrp    [1, 2, 0, 15, 0]
BCInFL      [1, 1, 0, 5, 0, 0, 0]
BCIInFL     [1, 1, 0, 1, 0, 0, 0, 1, 0]
CA$_2$      []
SeA         []
NA          []
RA          [1, 1, 0, 3, 0, 0]
IRA         [1, 1, 0, 2, 0, 0, 0, 10, 102, 4412, 4886349]
BRMod       []
Set         [1, 1, 1]
MonoUn      []
Unar        []
AAlg        []
BCI         [1, 2, 5]
RtQgrp      [1, 3, 44, 14022]
Qgrp        [1, 1, 5, 35, 1411]
MouQgrp     [1, 1, 5]
Lp          [1, 1, 1, 2, 6, 109, 23746, 106228849, 9365022303540, 20890436195945769617, 1478157455158044452849321016]
MouLp

In [152]:
%%time
ch=jsl
def compareandupdatefs(chax,cl):
    li = finitemembers(cl)
    sli = [x for x in li if x <= 1000]
    n = min(max(len(sli),3),6)
    fs = finespectrum(chax,cl,n,True)
    print(cl,li)
    return li[:n]==fs, fs

for cl in m[54:97]:
    if fulldefinition(cl)!="none":
        fl = compareandupdatefs(ch,cl)
        if not fl[0]:
            print("***********",fl)
            latex_st = set_finitemembers(cl,fl[1],latex_st)

['(((x v y) v z)=(x v (y v z)))', '((x v y)=(y v x))', '((x v x)=x)', '((x<=y) <-> ((x v y)=y))']
Number of nonisomorphic models of cardinality 2  is  1


Number of nonisomorphic models of cardinality 3  is  2
Number of nonisomorphic models of cardinality 4  is  5


Number of nonisomorphic models of cardinality 5  is  15


Number of nonisomorphic models of cardinality 6  is  53
Jslat [1, 1, 2, 5, 15, 53, 222, 1078, 5994, 37622, 262776, 2018305, 16873364, 152233518, 1471613387, 15150569446, 165269824761]
['(c=c)']
Number of nonisomorphic models of cardinality 2  is  2
Number of nonisomorphic models of cardinality 3  is  5


Number of nonisomorphic models of cardinality 4  is  16


...

Number of nonisomorphic models of cardinality 5  is  36


Number of nonisomorphic models of cardinality 6  is  202
CIdRJSgrp [1, 1, 2, 8, 36, 202]
['((x<=y) -> ((x*z)<=(y*z)))', '((x<=y) -> ((z*x)<=(z*y)))', '(((x*y)*z)=(x*(y*z)))', '((x*1)=x)', '((1*x)=x)', '(((x*y)<=z) <-> (y<=(x\\ z)))', '(((x*y)<=z) <-> (x<=(z/y)))', '((x*x)=x)', '((x*y)=(y*x))']
Number of nonisomorphic models of cardinality 2  is  1


Number of nonisomorphic models of cardinality 3  is  2
Number of nonisomorphic models of cardinality 4  is  6


Number of nonisomorphic models of cardinality 5  is  20


Number of nonisomorphic models of cardinality 6  is  77
CIdRJMon [1, 1, 2, 6, 20, 77, 333]
CPU times: user 8.27 s, sys: 11.9 s, total: 20.2 s
Wall time: 3min 48s


In [9]:
fulldefinition("Bnd")

[]

In [71]:
len(latex_st)

121318

In [72]:
writefile("SOASuo.tex",latex_st)

'Wrote 121318 characters'

In [9]:
print(finitememberslatex(fs))

$f_1 = 1$,
$f_2 = 3$,
$f_3 = 17$,
$f_4 = 138$,
$f_5 = 1555$



In [61]:
%%time
fs=finespectrum([],'Qnd',5,True)

['((x r (y r z))=((x r y) r (x r z)))', '(((x t y) t z)=((x t z) t (y t z)))', '(((x r y) t x)=y)', '((x r (y t x))=y)', '((x r x)=x)', '((x t x)=x)']
Number of nonisomorphic models of cardinality 2  is  1


Number of nonisomorphic models of cardinality 3  is  3
Number of nonisomorphic models of cardinality 4  is  7


Number of nonisomorphic models of cardinality 5  is  22
CPU times: user 20.3 ms, sys: 220 ms, total: 240 ms
Wall time: 517 ms


In [19]:
a=p9(jsl+['(((x->y)->((y->z)->(x->z)))=1)', '((1->x)=x)', '((x->1)=1)', '((x->(x v y))=1)', '(x<=((x->y)->y))'],[],1000,1000,[6])

Number of nonisomorphic models of cardinality 2  is  1
Number of nonisomorphic models of cardinality 3  is  3
Number of nonisomorphic models of cardinality 4  is  14


Number of nonisomorphic models of cardinality 5  is  87


Number of nonisomorphic models of cardinality 6  is  745
Fine spectrum:  [1, 1, 3, 14, 87, 745]


In [31]:
Po=[[],[
'Pos', #po-algs 10
'pPos',
'PoUn',
'PoNUn',
'PoMag',
'PoSgrp',
'PoMon',
'IPoMon',
'IdPoSgrp',
'IdPoMon',
'PoImpA', #implicative, left-residuated 7
'LrPoMag',
'LrPoSgrp',
'LrPoMon',
'Polrim',
'IdLrPoSgrp',
'IdLrPoMon',
'RPoUn', #residuated 8
'DivPos',
'RPoMag',
'RPoSgrp',
'RPoMon',
'Porim',
'IdRPoSgrp',
'IdRPoMon',
'GalPos', #Galois, involutive 6
'InPos',
'InPoMag',
'InPoSgrp',
'InPoMon',
'InPorim',
#'IdInPoSgrp',
#'IdInPoMon',
'CyInPoMag', #cyclic involutive 5
'CyInPoSgrp',
'CyInPoMon',
'CyInPorim',
#'CyIdInPoSgrp',
#'CyIdInPoMon',
'PoGrp',
'CPoSgrp', #commutative po-algs 5
'CPoMon',
'CIPoMon',
'CIdPoSgrp',
'CIdPoMon',
'CDivPos', #commutative residuated 8
'BCK',
'CRPoMag',
'CRPoSgrp',
'CRPoMon',
'Pocrim',
'CIdRPoSgrp',
'CIdRPoMon',
'CInPoMag', # commutative involutive 5
'CInPoSgrp',
'CInPoMon',
'InPocrim',
'APoGrp',
], #total 54 categories of po-algebras
['Jslat',
 'pJslat',
 'lbJslat',
 'ubJslat',
 'JUn',
 'JNUn',
 'GalJslat',

 'JMag',
 'JSgrp',
 'JMon',
 'IdSRng$_0$',
 'IdSRng$_{01}$',
 'KA',
 'IJMon',
 'IdJSgrp',
 'IdJMon',

 'JImpA',
 'LrJMag',
 'LrJSgrp',
 'LrJMon',
 'ILrJMon',
 'IdLrJSgrp',
 'IdLrJMon',

 'DivJslat',
 'RJMag',
 'RJSgrp',
 'RJMon',
 'IRJMon',
 'IdRJSgrp',
 'IdRJMon',
 
 'CJSgrp',
 'CJMon',
 'CIJMon',
 'CIdJSgrp',
 'CIdJMon',

 'CDivJslat',
 'BCKJslat',
 'CRJMag',
 'CRJSgrp',
 'CRJMon',
 'CIRJMon',
 'CIdRJSgrp',
 'CIdRJMon',
],
['Mslat',
 'pMslat',
 'MUn',
 'MNUn',
 'GalMslat',

 'MMag',
 'MSgrp',
 'MMon',
 'IMMon',
 'IdMSgrp',
 'IdMMon',

 'MImpA',
 'LrMMag',
 'LrMSgrp',
 'LrMMon',
 'ILrMMon',
 'RtHp',
 'IdLrMSgrp',
 'IdLrMMon',

 'DivMslat',
 'RMMag',
 'RMSgrp',
 'RMMon',
 'IRMMon',
 'IdRMSgrp',
 'IdRMMon',

 'CMSgrp',
 'CMMon',
 'CIMMon',
 'CIdMSgrp',
 'CIdMMon',

 'CDivMslat',
 'BCKMslat',
 'CRMMag',
 'CRMSgrp',
 'CRMMon',
 'CIRMMon',
 'Hp',
 'CIdRMSgrp',
 'CIdRMMon',
 'HilA',
 'BrSlat',
],
['Lat',
 'pLat',
 'bLat',
 '$\\ell$Un',
 '$\\ell$NUn',

 '$\\ell$Mag',
 '$\\ell$Sgrp',
 '$\\ell$Mon',
 'KLat',
 'MLat',
 'MultLat',
 'I$\\ell$Mon',
 'Id$\\ell$Sgrp',
 'Id$\\ell$Mon',

 '$\\ell$ImpA',
 'Lr$\\ell$Mag',
 'Lr$\\ell$Sgrp',
 'Lr$\\ell$Mon',
 'ILr$\\ell$Mon',
 'IdLr$\\ell$Sgrp',
 'IdLr$\\ell$Mon',

 'DivLat',
 'R$\\ell$Mag',
 'R$\\ell$Sgrp',
 'RL',
 'bRLat',
 'IRL',
 'IdR$\\ell$Sgrp',
 'IdRL',
 'FL',
 'FL$_c$',
 'FL$_e$',
 'FL$_w$',
 'FL$_{ec}$',
 'FL$_{ew}$',

 'GalLat',
 'InLat',
 'In$\\ell$Mag',
 'In$\\ell$Sgrp',
 'InFL',
 'IInFL',

 'CyIn$\\ell$Mag',
 'CyIn$\\ell$Sgrp',
 'CyInFL',
 'CyIInFL',

 'C$\\ell$Sgrp',
 'C$\\ell$Mon',
 'CI$\\ell$Mon',
 'CId$\\ell$Sgrp',
 'CId$\\ell$Mon',

 'CDivLat',
 'BCKlat',
 'CR$\\ell$Mag',
 'CR$\\ell$Sgrp',
 'CRL',
 'CIRL',
 'CIdR$\\ell$Sgrp',
 'CIdRL',
 'CIdInFL',

 'CIn$\\ell$Mag',
 'CIn$\\ell$Sgrp',
 'CInFL',
 'CIInFL',

 'SkLat',
 'JsdLat',
 'MsdLat',
 'SdLat',
 'NdLat',
 'ADLat',

 'CdLat',
 'OLat',
 'OMLat',
 'MOLat',
 'Bilat',

 'CanRL',
 'CdMLat',
 'FRng',
 'ILLA',
 'LLA',
 'MALLA',
 'psMV',
]
]

In [0]:
Po=['DLat',
 'pDLat',
 'BDLat',
 'D$\\ell$Un',
 'D$\\ell$NUn',
 'pcDLat',
 'OckA',

 'DeMA',
 'DMMon',
 'DpAlg',
 'DdpAlg',
 'DDblpAlg',
 'StAlg',
 'DblStAlg',

 'D$\\ell$Mag',
 'D$\\ell$Sgrp',
 'D$\\ell$Mon',
 'DI$\\ell$Mon',
 'DId$\\ell$Sgrp',
 'DId$\\ell$Mon',

 'D$\\ell$ImpA',
 'DLr$\\ell$Mag',
 'DLr$\\ell$Sgrp',
 'DLr$\\ell$Mon',
 'DILr$\\ell$Mon',
 'DIdLr$\\ell$Sgrp',
 'DIdLr$\\ell$Mon',

 'DDivLat',
 'DR$\\ell$Mag',
 'DR$\\ell$Sgrp',
 'DRL',
 'DIRL',
 'DIdR$\\ell$Sgrp',
 'DIdRL',

 'DGalLat',
 'DInLat',
 'DIn$\\ell$Mag',
 'DIn$\\ell$Sgrp',
 'DInFL',
 'DIInFL',

 'CyDIn$\\ell$Mag',
 'CyDIn$\\ell$Sgrp',
 'CyDInFL',
 'CyDIInFL',
 '$\\ell$Grp',
 'Rep$\\ell$Grp',

 'CD$\\ell$Sgrp',
 'CD$\\ell$Mon',
 'CDI$\\ell$Mon',
 'CDId$\\ell$Sgrp',
 'CDId$\\ell$Mon',

 'CDDivLat',
 'CDR$\\ell$Mag',
 'CDR$\\ell$Sgrp',
 'CDRL',
 'CDIRL',
 'CDIdR$\\ell$Sgrp',
 'CDIdRL',

 'CDIn$\\ell$Mag',
 'CDIn$\\ell$Sgrp',
 'CDInFL',
 'CDIInFL',
 'A$\\ell$Grp',

 'GBL',
 'GMV',
 'WHp',
 'BrA',
 'GBA',
 'BoolLat',

 'CRSlSgrp',
 'CRSlMon',
 'CIRSlMon',
 'MTLA',
 'BLA',
 'MV',
 'HA',
 'GödA',

 'CIdRSlSgrp',
 'CIdRSlMon',
 'CInSlSgrp',
 'CInSlMon',
 'DunnMon',
 'CIInRSlMon',

 'ImpLat',
 'KLA',
 'NVLGrp',
 'LA$_n$',
 'MZrd',
 '$\\ell$Rng',
 'ORng',
 'C$\\ell$Rng',
 'CORng',
 'OFld',

 #'DLE',
 #'DLOS',
 #'SlMag',
 #'SlSgrp',
 #'CSlSgrp',
 #'SlMon',
 #'CSlMon',
 #'ISlMon',
 #'CISlMon',
 #'IdSlSgrp',
 #'CIdSlSgrp',
 #'IdSlMon',
 #'CIdSlMon',
 #'SlImpA',
 #'LrSlMag',
 #'LrSlSgrp',
 #'LrSlMon',
 #'ILrSlMon',
 #'IdLrSlSgrp',
 #'IdLrSlMon',
 #'SlDivLat',
 #'CSlDivLat',
 #'RSlMag',
 #'CRSlMag',
 #'RSlSgrp',
 #'RSlMon',
 #'IRSlMon',
 #'IdRSlSgrp',
 #'IdRSlMon',
 #'SlUn',
 #'SlNUn',
 #'SlGalLat',
 #'SlInLat',
 #'InSlMag',
 #'CyInSlMag',
 #'CInSlMag',
 #'InSlSgrp',
 #'CyInSlSgrp',
 #'InSlMon',
 #'CyInSlMon',
 #'IInRSlMon',
 #'CyIInRSlMon',
 #'Chain',
 #'OSgrp',
 #'COSgrp',
 #'OMon',
 #'OMonZ',
 #'COMon',
 #'OGrp',
 #'OAbGrp',
 #'OSlat',
 #'RLGrp',

 'ToLat', #Chn
 'pToLat',
 'ToMag',
 'ToSgrp',
 'ToMon',
 'IToMon',
 'IdToSgrp',
 'IdToMon',

 'ToImpA',
 'LrToMag',
 'LrToSgrp',
 'LrToMon',
 'ILrToMon',
 'IdLrToSgrp',
 'IdLrToMon',

 'DivChn',
 'RToMag',
 'RToSgrp',
 'RToMon',
 'IRToMon',
 'IdRToSgrp',
 'IdRToMon',

 'ToUn',
 'ToNUn',
 'GalChn',
 'InChn',
 'InToMag',
 'InToSgrp',
 'InToMon',
 'IInRToMon',
 'CyInToMag',
 'CyInToSgrp',
 'CyInToMon',
 'CyIInRToMon',
 'ToGrp',

 'CToSgrp',
 'CToMon',
 'CIToMon',
 'CIdToSgrp',
 'CIdToMon',

 'CDivChn',
 'CRToMag',
 'CRToSgrp',
 'CRToMon',
 'CIRToMon',
 'CIdRToSgrp',
 'CIdRToMon',

 'CInToMag',
 'CInToSgrp',
 'CInToMon',
 'CIInRToMon',
 'AbToGrp',

 'BA',
 'pBA',
 'BUn',
 'BNUn',
 'MA',
 'TA',
 'MonA',

 'BMag',
 'BSgrp',
 'BMon',
 'BIMon',
 'BIdSgrp',
 'BIdMon',

 'BImpA',
 'BLrMag',
 'BLrSgrp',
 'BLrMon',
 'BILrMon',
 'BIdLrSgrp',
 'BIdLrMon',

 'BDivLat',
 'BRMag',
 'BRSgrp',
 'BRL',
 'BIRL',
 'BIdRSgrp',
 'BIdRL',

 'BGalLat',
 'BInLat',
 'BInMag',
 'BInSgrp',
 'BInFL',
 'BIInFL',

 'BCyInMag',
 'BCyInSgrp',
 'BCyInFL',
 'BCyIInFL',

 'BCSgrp',
 'BCMon',
 'BCIMon',
 'BSlat',
 'BCIdSgrp',
 'BCIdMon',

 'BCDivLat',
 'BCRMag',
 'BCRSgrp',
 'BCRL',
 'BCIRL',
 'BCIdRSgrp',
 'BCIdRL',

 'BCInMag',
 'BCInSgrp',
 'BCInFL',
 'BCIInFL',

 'CA$_2$',
 'SeA',
 'NA',
 'RA',
 'IRA',
 'DGrp',
 'BRMod',

 #'BAO',
 #'BSp',
 #'CanPMon',
 'MonoUn',
 'Unar',
 'AAlg',
 'BCI',

 'RtQgrp',
 'Qgrp',
 'MouQgrp',
 'Lp',
 'MouLp',

 'Shells',
 'Mag',
 'Bnd',
 'NBnd',
 'RBnd',

 'Sgrp',
 'Sgrp$_0$',
 'RSgrp',
 'InvSgrp',
 'Mon',
 'CanSgrp',
 'CanMon',
 'Grp',
 'ApGrp',

 'CMag',
 'CSgrp',
 'LCanSgrp',
 'CanCSgrp',
 'CInvSgrp',
 'CMon',
 'CanCMon',
 'AbGrp',

 'NFld',
 'NRng',
 'NRng$_1$',
 'Nfld',

 'SRng',
 'SRng$_1$',
 'SRng$_0$',
 'SRng$_{01}$',
 'Rng',
 'Rng$_1$',
 'RRng',

 'CRRng',
 'CRng',
 'CRng$_1$',
 'IntDom',
 'DRng',
 'Sfld',
 'Fld',

 'ConMag',
 'Dtoid', #move to Po?
 'UFDom',
 'OreDom',
 'PIDom',
 'EucDom',

 'MSet',
 'Gset',
 'RMod',
 'FVec',

 'JorA',
 'LNfld',
 'BilinA',
 'CliffSgrp',
 'LieA',
 'MedMag',
 'NlGrp',
 'NaA',
 'OrdA',
 'p-Grp',
 'Qnd',
 'qMV',
 'sqMV',
 'QtMag',
 'Set',
]

In [0]:
[ 'MonoUn',
 'Unar',
 'BCI',

 'RtQgrp',
 'Qgrp',
 'MouQgrp',
 'Lp',
 'MouLp',

 'Shells',
 'Mag',
 'Bnd',
 'NBnd',
 'RBnd',

 'Sgrp',
 'Sgrp$_0$',
 'RSgrp',
 'InvSgrp',
 'Mon',
 'CanSgrp',
 'CanMon',
 'Grp',
 'ApGrp',

 'CMag',
 'CSgrp',
 'LCanSgrp',
 'CanCSgrp',
 'CInvSgrp',
 'CMon',
 'CanCMon',
 'AbGrp',

 'NFld',
 'NRng',
 'NRng$_1$',
 'Nfld',

 'SRng',
 'SRng$_1$',
 'SRng$_0$',
 'SRng$_{01}$',
 'Rng',
 'Rng$_1$',
 'RRng',

 'CRRng',
 'CRng',
 'CRng$_1$',
 'IntDom',
 'DRng',
 'Sfld',
 'Fld',

 'ConMag',
 'Dtoid', #move to Po?
 'UFDom',
 'OreDom',
 'PIDom',
 'EucDom',

 'MSet',
 'Gset',
 'RMod',
 'FVec',
 'NaA',
 'BilinA',
 'LieA',
 'JorA',
 'AAlg',

 'LNfld',
 'CliffSgrp',
 'MedMag',
 'NlGrp',

 'OrdA',
 'p-Grp',
 'Qnd',
 'qMV',
 'sqMV',
 'QtMag',
 'Set',
]

In [30]:
m

['Mslat',
 'pMslat',
 'MMag',
 'MSgrp',
 'CMSgrp',
 'MMon',
 'CMMon',
 'IMMon',
 'CIMMon',
 'IdMSgrp',
 'CIdMSgrp',
 'IdMMon',
 'CIdMMon',
 'MImpA',
 'LrMMag',
 'LrMSgrp',
 'LrMMon',
 'ILrMMon',
 'IdLrMSgrp',
 'IdLrMMon',
 'DivMslat',
 'CDivMslat',
 'RMMag',
 'CRMMag',
 'RMSgrp',
 'CRMSgrp',
 'RMMon',
 'CRMMon',
 'IRMMon',
 'CIRMMon',
 'IdRMSgrp',
 'CIdRMSgrp',
 'IdRMMon',
 'CIdRMMon',
 'MUn',
 'MNUn',
 'GalMslat',
 'Hp',
 'RtHp',
 'BrSlat',
 'BCKMslat',
 'HilA']

In [49]:
cl=re.findall(r"\\subsection{(.*?)}",st,flags=re.DOTALL)
cl

['Partially ordered sets',
 'Pointed posets',
 'Partially ordered unars',
 'Partially ordered semigroups',
 'Commutative partially ordered semigroups',
 'Commutative residuated partially ordered semigroups',
 'Partially ordered monoids',
 'Commutative partially ordered monoids',
 'Residuated partially ordered monoids',
 'Commutative residuated partially ordered monoids',
 'Pocrims',
 'Right hoops',
 'Partially ordered groups',
 'Abelian partially ordered groups',
 'Hilbert algebras',
 'Semilattices',
 'Semilattices with identity',
 'Semilattices with zero',
 'Multiplicative semilattices',
 'BCK-join-semilattices',
 'Idempotent semirings',
 'Idempotent semirings with identity',
 'Idempotent semirings with zero',
 'Idempotent semirings with identity and zero',
 'Kleene algebras',
 'Hoops',
 'Brouwerian semilattices',
 'BCK-meet-semilattices',
 'Action lattices',
 'Algebraic Lattices',
 'Almost distributive lattices',
 'BCK-lattices',
 'Bilattices',
 'Bounded lattices',
 'Bounded residuat

In [50]:
len(cl)

234

In [52]:
de=re.findall(r"\\begin{definition}(.*?)\\end{definition}",st,flags=re.DOTALL)

In [55]:
print(de[6])


A \emph{commutative residuated partially ordered semigroup} is a ((residuated partially ordered semigroup)) $\mathbf{A}=\langle A, \cdot, \to, \le\rangle$ such that

$\cdot$ is \emph{commutative}:  $xy=yx$





In [17]:
n=0
n=st1.find("\\begin{definition}",n+1)
st2=st1[:n]+st1[n:].replace("==","\\end{defintion}\n==",1)

527

In [30]:
import re
m=re.search("begin{definition}.*?==",st1,flags=re.DOTALL)
m

<re.Match object; span=(1125, 1653), match="begin{definition}\nAn \\emph{abelian partially or>

In [75]:
def rep(m): return "\\begin{tabular}{|ll|}\\\\hline\n^[[Classtype"
st1=re.sub("\^(\[\[Classtype)",rep,st,flags=re.DOTALL)

In [71]:
m=re.search("\^(\[\[Classtype)",st,flags=re.DOTALL)
m

<re.Match object; span=(2328, 2340), match='^[[Classtype'>

In [68]:
print(st[2300:3100])

s that is being described.

^[[Classtype]]                        |quasivariety [(Lastname19xx)]  |
^[[Equational theory]]                | |
^[[Quasiequational theory]]           | |
^[[First-order theory]]               | |
^[[Locally finite]]                   | |
^[[Residual size]]                    | |
^[[Congruence distributive]]          | |
^[[Congruence modular]]               | |
^[[Congruence $n$-permutable]]        | |
^[[Congruence regular]]               | |
^[[Congruence uniform]]               | |
^[[Congruence extension property]]    | |
^[[Definable principal congruences]]  | |
^[[Equationally def. pr. cong.]]      | |
^[[Amalgamation property]]            | |
^[[Strong amalgamation property]]     | |
^[[Epimorphisms are surjective]]      | |

====Finite members====

$\b


In [20]:
def rep(m): return "\\begin{tabular}{|ll|}\\\\hline\n^[[Classtype"
st1=re.sub("\^(\[\[Classtype)",rep,st,flags=re.DOTALL)

In [21]:
def rep(m): return "hline\n\\end{tabular}\n\n\n\\begin{finite"
st1=re.sub("begin{finite",rep,st1,flags=re.DOTALL)

In [22]:
def rep(m): return m.group(1)+"&"+m.group(2)+"\\\\"
st1=re.sub("\^(\[\[.*?)\|(.*?)\|",rep,st1,flags=re.DOTALL)

In [23]:
st1=st1.replace("[[","((").replace("]]","))")

In [18]:
len(st1)

2

In [17]:
print(st1[2000:3500])

()


In [34]:
m=re.search(r"=====(.*?)=====.*?Abbreviation: \*\*(.*?)\*\*",st1,flags=re.DOTALL)
m.group(1), m.group(2), m

('BCK-join-semilattices',
 'BCKJSlat',
 <re.Match object; span=(0, 59), match='=====BCK-join-semilattices=====\n\nAbbreviation: >)

In [36]:
def rep(m): return "\n\\hrulefill\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\\hypertarget{"+m.group(2)+"}{\\subsection{"+m.group(1)+"}}\\ \n\n\\abbreviation{"+m.group(2)+"}"
st2=re.sub(r"=====(.*?)=====.*?Abbreviation: \*\*(.*?)\*\*",rep,st1,flags=re.DOTALL)

In [37]:
len(st2)

23967

In [38]:
print(st2[3000:5200])

&Variety \\
((Equational theory))  &Decidable \\
((Quasiequational theory))  & \\
((First-order theory))  &Undecidable \\
((Locally finite))  &No \\
((Residual size))  &Unbounded \\
((Congruence distributive))  &No \\
((Congruence modular))  &No \\
((Congruence n-permutable))  & \\
((Congruence regular))  & \\
((Congruence uniform))  & \\
((Congruence extension property))  & \\
((Definable principal congruences))  & \\
((Equationally def. pr. cong.))  & \\
((Amalgamation property))  & \\
((Strong amalgamation property))  & \\
((Epimorphisms are surjective))  & \\
\hline
\end{tabular}


\begin{finitemembers}

$\begin{array}{lr}
f(1)= &1\\
f(2)= &6\\
f(3)= &61\\
f(4)= &866\\
f(5)= &\\
f(6)= &\\
\end{array}$

\end{finitemembers}


\begin{subclasses}
((Idempotent semirings with identity)) 

((Idempotent semirings with zero)) 

\end{subclasses}


\begin{superclasses}
((Semirings)) 


\end{superclasses}


)]
\hrulefill
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\hypertarget{ISRng$_1$

In [6]:
def extract(st, env):
    # find all environments in st named env and print them
    li=re.findall(r"(\\begin{"+env+r"}.*?\\end{"+env+r"})",st,flags=re.DOTALL)
    return li
a = extract(st,"examples")

In [7]:
len(a)

248

In [8]:
a[:20]

['\\begin{examples}\nExample 1: \n\n\\end{examples}',
 '\\begin{examples}\nExample 1: \n\n\\end{examples}',
 '\\begin{examples}\nExample 1: \n\n\\end{examples}',
 '\\begin{examples}\nExample 1: \n\n\\end{examples}',
 '\\begin{examples}\nExample 1: \n\n\\end{examples}',
 '\\begin{examples}\nExample 1: \n\n\\end{examples}',
 '\\begin{examples}\nExample 1: \n\n\\end{examples}',
 '\\begin{examples}\nExample 1: Given any poset with top element 1, $\\langle A,\\le, 1\\rangle$, define $a\\to b=\\begin{cases}1&\\text{ if $a\\le b$}\\\\ b&\\text{ otherwise.}\\end{cases}$ Then $\\langle A,\\to,1\\rangle$ is a Hilbert algebra.\n\\end{examples}',
 '\\begin{examples}\nExample 1: The integers, the rationals and the reals with the usual order.\n\n\\end{examples}',
 '\\begin{examples}\nExample 1: \n\n\\end{examples}',
 '\\begin{examples}\nExample 1: The natural numbers larger than 1, with addition, or with multiplication.\n\n\\end{examples}',
 '\\begin{examples}\nExample 1: $\\langle \\mathbb{R},\\leq

NameError: name 'sage' is not defined